# Conservative Q-Learning for Statistical Arbitrage

This notebook demonstrates how to configure and train a Conservative Q-Learning (CQL) agent for statistical arbitrage using observations and actions from a simple rule-based strategy. The CQL agent learns to optimize expected returns using offline reinforcement learning.

## Install Packages

The following libraries are required for this notebook. If you haven't installed them yet, you can do so using running the cell below or by using pip install in your terminal.

In [1]:
# Install required packages including d3rlpy
# %pip -q install -U numpy pandas pyarrow gdown gymnasium torch matplotlib tensorboard d3rlpy

## Import Libraries

In [2]:
import os
from gdown import download
import random

from typing import Tuple, Set, Optional, Match
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

# Reinforcement Learning - d3rlpy
import gymnasium as gym
import d3rlpy

# Deep Learning
import torch

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


## Configuration

The configuration section sets up the parameters for the reinforcement learning model.

In [3]:
CONFIG = {
    "DATA": {
        "forward_fill": True,
        "drop_na_after_ffill": True,
        "cache_dir": "./data_cache",
        "timestamp_format": "%Y-%m-%d %H:%M:%S",
        "asset_price_format": "{ASSET}_{FEATURE}",
        "pair_feature_format": "{ASSET1}_{ASSET2}_{FEATURE}",
        "timestamp_col": "timestamp",
        "sampling": "1m",
        "features": {
            "file_id": "1OCqEkOWV73Z8e-67fpqVL3r3ugVcfml8",
            "file_name": "bin_futures_full_features",
            "type": "csv",
            "seperator": ",",
            "index": "datetime",
            "start": "2024-05-01 00:00:00",
            "end": "2025-05-01 00:00:00",
            "individual_identifier": "close",
            "pair_identifier": "beta",
        },
    },
    "Strategy": {
        "z-score": "spreadNorm",
        "entry": 1.5,
        "exit": 0.5,
    },
    "BC": {
        "batch_size": 256,
        "n_epochs": 50,
        "learning_rate": 3e-4,
        "gamma": 0.99,
        "tau": 0.005,
        "n_critics": 2,
        "use_gpu": True,
    },
    "CQL": {
        "batch_size": 256,
        "n_steps": 10000,
        "n_steps_per_epoch": 1000,
        "learning_rate": 3e-4,
        "gamma": 0.99,
        "tau": 0.005,
        "alpha": 5.0,
        "conservative_weight": 5.0,
        "use_gpu": True,
        "encoder_hidden_sizes": [256, 256],
        "q_func_hidden_sizes": [256, 256],
    },
    "CQL_PnL": {
        "batch_size": 256,
        "n_steps": 10000,
        "n_steps_per_epoch": 1000,
        "learning_rate": 3e-4,
        "gamma": 0.99,
        "tau": 0.005,
        "alpha": 5.0,
        "use_gpu": True,
        "encoder_hidden_sizes": [256, 256],
        "q_func_hidden_sizes": [256, 256],
        "transaction_cost_bps": 3.5,  # 0.035% = 3.5 basis points
    },
    "ENV": {
        "seed": 42,
        "trading_window_days": "2D",
        "sliding_window_step": "1D",
    },
    "SPLITS": {
        "data_start": "2024-05-01",
        "data_end": "2025-04-30",
        "train": ["2024-05-01", "2024-12-31"],    # ~8 months (70%)
        "val":   ["2025-01-01", "2025-02-28"],    # ~2 months (15%)
        "test":  ["2025-03-01", "2025-04-30"],    # ~2 months (15%)
    },
    "EVAL": {
        "plots": True,
        "reports_dir": "./reports"
    },
    "IO": {
        "models_dir": "./models",
        "tb_logdir": "./tb_logs",
        "dataset_dir": "./offline_datasets"
    }
}

## Helper Functions

In [4]:
# set annualization factors for different timeframes
ANNUALIZATION = {"1m":365*24*60,
                 "5m":365*24*12,
                 "15m":365*24*4,
                 "1h":365*24,
                 "1d":365}

# create directory if it doesn't already exist
def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

## Set Computation Device

This section sets the computation device for training the model. It checks if a GPU is available and sets it as the device; otherwise, it defaults to CPU.

In [5]:
# run on cuda GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")

# run on Apple Silicon GPU
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS is available. Using Apple Silicon GPU.")

# run on CPU (slow)
else:
    device = torch.device("cpu")
    print("CUDA and MPS are not available. Using CPU.")

MPS is available. Using Apple Silicon GPU.


## Set Seeds

This section sets the random seeds for various libraries to ensure that the results are reproducible.

Note: It is good practice to type (set data types) for function and method parameters for better code maintainability.

In [6]:
def set_all_seeds(seed: int = 42):
    random.seed(seed) # seed for random module
    np.random.seed(seed) # seed for numpy module
    try:
        torch.manual_seed(seed) # seed for torch module
        if torch.cuda.is_available(): # seed for CUDA device
            torch.cuda.manual_seed_all(seed)
        # Note: MPS uses the same manual_seed as regular torch
    except Exception:
        pass

set_all_seeds(CONFIG["ENV"]["seed"]) # set all seeds for reproducibility

## Download Feature Data

In this section, we retrieve the .csv file created during feature engineering.

In [7]:
def download_file(file_name: str, file_id: str, out_dir: str):
    ensure_dir(out_dir)

    out_path = os.path.join(out_dir, f"{file_name}.csv")  # full output path

    if os.path.exists(out_path):
        print(f"Skipping download. File {file_name} already exists in cache.")
        return

    try:
        print(f"Downloading {file_name} -> {out_path}")
        url = f"https://drive.google.com/uc?id={file_id}"
        success = download(url, out_path, quiet=False, use_cookies=False, verify=True)
        print("Download complete.")
        return success
    except Exception as e:
        print(f"Download attempt failed for {file_name}. Error: {str(e)}")
    
# donwload features
file_name = CONFIG["DATA"]["features"]["file_name"]
file_id = CONFIG["DATA"]["features"]["file_id"]
cache_dir = CONFIG["DATA"]["cache_dir"]

success = download_file(file_name, file_id, cache_dir)

Skipping download. File bin_futures_full_features already exists in cache.


## Load Feature Data

Once the data is downloaded, this section loads the data into a pandas DataFrame for later processing.

In [8]:
def load_csv_to_df(
    path: str,
    sep: str = ",",
    timestamp_index_col: str | None = "datetime",
    encoding: str = "utf-8-sig",
    **read_csv_kwargs,
) -> pd.DataFrame:
    """
    Load a CSV into a pandas DataFrame.

    Parameters
    ----------
    path : str
        Filesystem path to the CSV.
    parse_timestamp_col : str | None
        If provided and present in the CSV, this column will be parsed to datetime.
        Set to None to skip datetime parsing.
    **read_csv_kwargs :
        Extra arguments passed to `pd.read_csv` (e.g., sep, dtype, usecols).

    Returns
    -------
    pd.DataFrame
    """

    # Parse header-only to check for timestamp col presence
    head = pd.read_csv(path, sep=sep, encoding=encoding, nrows=0)
    if timestamp_index_col and timestamp_index_col in head.columns:
        read_csv_kwargs = {
            **read_csv_kwargs,
            "parse_dates": [timestamp_index_col],
        }

    df = pd.read_csv(path, sep=sep, encoding=encoding, engine="pyarrow", **read_csv_kwargs)

    df = df.set_index("datetime")

    return df


# load features
file_name = CONFIG["DATA"]["features"]["file_name"]
cache_dir = CONFIG["DATA"]["cache_dir"]
index = CONFIG["DATA"]["features"]["index"]
sep = CONFIG["DATA"]["features"].get("seperator", ",")
file_path = os.path.join(cache_dir, f"{file_name}.csv")
features_df = load_csv_to_df(file_path, sep, timestamp_index_col=index)

# print dataframe info
print("Features DataFrame Info:")
print(features_df.info())

Features DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 525600 entries, 2024-05-01 00:00:00 to 2025-04-30 23:59:00
Columns: 789 entries, AAVE_close to XRP_fundingMinutesLeft
dtypes: float64(763), int64(25), object(1)
memory usage: 3.1+ GB
None


## Identify Feature Structure

In [9]:
def identify_assets_features_pairs(
    df: pd.DataFrame,
    single_asset_format: str,
    pair_feature_format: str,
) -> Tuple[list[str], list[str], list[str], list[Tuple[str, str]]]:
    """
    Returns distinct
      1. assets
      2. single-asset feature names (ARB_closeUpperShadow → closeUpperShadow)
      3. pair feature names (ARB_ETH_spreadNorm → spreadNorm)
      4. unordered asset pairs
    """

    def format_to_regex(fmt: str) -> re.Pattern:
        escaped = re.escape(fmt)

        def repl(match: Match[str]) -> str:
            name = match.group(1)
            char_class = r"[A-Za-z0-9_]+" if "FEATURE" in name.upper() else r"[A-Za-z0-9]+"
            return f"(?P<{name}>{char_class})"

        escaped = re.sub(r"\\\{(\w+)\\\}", repl, escaped)
        return re.compile(f"^{escaped}$")

    single_asset_pattern = format_to_regex(single_asset_format)
    pair_feature_pattern = format_to_regex(pair_feature_format)
    generic_single_pattern = re.compile(r"^(?P<ASSET>[A-Za-z0-9]+)_(?P<FEATURE>[A-Za-z0-9_]+)$")

    assets: Set[str] = set()
    single_features: Set[str] = set()
    pair_features: Set[str] = set()
    pairs: Set[Tuple[str, str]] = set()

    literal_feature = None
    if "{FEATURE}" not in single_asset_format:
        literal_feature = single_asset_format.replace("{ASSET}", "").lstrip("_")

    skip_cols = {"timestamp", "datetime", "date"}

    for col in df.columns:
        if col in skip_cols:
            continue

        match_pair = pair_feature_pattern.match(col)
        if match_pair:
            a1, a2, feat = match_pair.group("ASSET1"), match_pair.group("ASSET2"), match_pair.group("FEATURE")
            assets.update((a1, a2))
            pairs.add(tuple(sorted((a1, a2))))
            pair_features.add(feat)
            continue

        match_single = single_asset_pattern.match(col)
        if match_single:
            asset = match_single.group("ASSET")
            assets.add(asset)
            feat = match_single.groupdict().get("FEATURE") or literal_feature
            if feat:
                single_features.add(feat)
            continue

        match_generic = generic_single_pattern.match(col)
        if match_generic:
            asset, feat = match_generic.group("ASSET"), match_generic.group("FEATURE")
            assets.add(asset)
            single_features.add(feat)
            continue

    return (
        sorted(assets),
        sorted(single_features),
        sorted(pair_features),
        sorted(pairs),
    )

# identify assets, features, and asset pairs
single_asset_format = CONFIG["DATA"]["asset_price_format"]
pair_feature_format = CONFIG["DATA"]["pair_feature_format"]
assets, single_asset_features, pair_features, asset_pairs = identify_assets_features_pairs(
    features_df,
    CONFIG["DATA"]["asset_price_format"],
    CONFIG["DATA"]["pair_feature_format"],
)

print(f"Identified {len(assets)} assets: {assets}")
print(f"Identified {len(asset_pairs)} asset pairs: {asset_pairs}")
print(f"Identified {len(single_asset_features)} single-asset features: {single_asset_features}")
print(f"Identified {len(pair_features)} pair features: {pair_features}")

Identified 25 assets: ['AAVE', 'ADA', 'APT', 'ARB', 'ATOM', 'AVAX', 'BCH', 'BNB', 'BTC', 'DOGE', 'DOT', 'ENA', 'ETC', 'ETH', 'HBAR', 'LINK', 'LTC', 'NEAR', 'SUI', 'TON', 'TRX', 'UNI', 'WLD', 'XLM', 'XRP']
Identified 61 asset pairs: [('AAVE', 'SUI'), ('AAVE', 'TRX'), ('ADA', 'BTC'), ('ADA', 'DOGE'), ('ADA', 'HBAR'), ('ADA', 'LTC'), ('ADA', 'SUI'), ('ADA', 'XLM'), ('ADA', 'XRP'), ('APT', 'AVAX'), ('ARB', 'ATOM'), ('ARB', 'AVAX'), ('ARB', 'DOT'), ('ARB', 'ETC'), ('ARB', 'ETH'), ('ARB', 'NEAR'), ('ARB', 'WLD'), ('ATOM', 'BCH'), ('ATOM', 'DOT'), ('ATOM', 'ENA'), ('ATOM', 'ETC'), ('AVAX', 'BCH'), ('AVAX', 'DOT'), ('AVAX', 'ETC'), ('AVAX', 'UNI'), ('BCH', 'DOT'), ('BCH', 'ENA'), ('BCH', 'ETC'), ('BNB', 'LINK'), ('BTC', 'DOGE'), ('BTC', 'HBAR'), ('BTC', 'LTC'), ('BTC', 'SUI'), ('BTC', 'TRX'), ('BTC', 'XLM'), ('BTC', 'XRP'), ('DOGE', 'LINK'), ('DOGE', 'LTC'), ('DOGE', 'SUI'), ('DOGE', 'XLM'), ('DOT', 'ENA'), ('DOT', 'ETC'), ('DOT', 'ETH'), ('ENA', 'ETC'), ('ENA', 'UNI'), ('ETC', 'ETH'), ('ETC',

## Build Time Intervals

In [10]:
def build_time_intervals(
    df: pd.DataFrame,
    window: pd.Timedelta | str,
    step: Optional[pd.Timedelta | str] = None,
    timestamp_col: str = "datetime",
    include_last_partial: bool = False,
) -> list[tuple[pd.Timestamp, pd.Timestamp]]:
    """
    Return fixed-length time intervals over the DataFrame's time span.

    Parameters
    ----------
    df : pd.DataFrame
        Must contain a datetime-like 'timestamp' column or have a DatetimeIndex.
    window : pd.Timedelta | str
        Size of each window, e.g. '2D', '60min', '15T'.
    step : pd.Timedelta | str | None
        Step between consecutive window starts. Defaults to `window` (non-overlapping).
        Use a smaller step than `window` for sliding/overlapping windows.
    timestamp_col : str
        Name of the timestamp column (ignored if index is a DatetimeIndex).
    include_last_partial : bool
        If True, include the trailing partial window shorter than `window`.

    Returns
    -------
    list[tuple[pd.Timestamp, pd.Timestamp]]
        Half-open intervals [start, end).
    """
    W = pd.Timedelta(window)
    S = pd.Timedelta(step) if step is not None else W

    # Extract, sanitize, and sort timestamps
    if timestamp_col in df.columns:
        ts = pd.to_datetime(df[timestamp_col]).dropna().sort_values()
    elif isinstance(df.index, pd.DatetimeIndex):
        ts = pd.Series(df.index).dropna().sort_values()
    else:
        raise ValueError(
            f"Timestamp column '{timestamp_col}' not found and index is not DatetimeIndex."
        )

    intervals: list[tuple[pd.Timestamp, pd.Timestamp]] = []
    if ts.empty:
        return intervals

    t_min = ts.iloc[0]
    t_max = ts.iloc[-1]
    cur = t_min

    while cur < t_max:
        end = cur + W
        if end <= t_max:
            intervals.append((cur, end))
        elif include_last_partial:
            intervals.append((cur, t_max))
            break
        else:
            break
        cur = cur + S

    return intervals

# build intervals
window = CONFIG["ENV"]["trading_window_days"]
step = CONFIG["ENV"]["sliding_window_step"]
timestamp_col = CONFIG["DATA"]["timestamp_col"]

intervals = build_time_intervals(
    features_df,
    window,
    step,
    timestamp_col,
    include_last_partial=False
)

# print interval info
print(f"Built {len(intervals)} time intervals with window={window} and step={step}.")
print("First 3 intervals:")
for start, end in intervals[:3]:
    print(f"  {start} to {end}")

Built 363 time intervals with window=2D and step=1D.
First 3 intervals:
  2024-05-01 00:00:00 to 2024-05-03 00:00:00
  2024-05-02 00:00:00 to 2024-05-04 00:00:00
  2024-05-03 00:00:00 to 2024-05-05 00:00:00


## Identify Feature Space

In [11]:
def is_timeframe_valid(
    df: pd.DataFrame,
    pair: tuple[str, str],
    start: pd.Timestamp,
    end: pd.Timestamp,
    feature_name: str,
    pair_feature_format: str = "{ASSET1}_{ASSET2}_{FEATURE}",
    timestamp_col: str | None = "datetime",
) -> bool:
    """
    Check if the given time frame has complete data for the specified asset pair.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the data.
    pair : tuple[str, str]
        Asset pair (asset1, asset2).
    start : pd.Timestamp
        Start of the time frame (inclusive).
    end : pd.Timestamp
        End of the time frame (exclusive).
    pair_feature_format : str
        Format string for pair-feature columns (e.g., "{ASSET1}_{ASSET2}_{FEATURE}").
    timestamp_col : str
        Name of the timestamp column.

    Returns
    -------
    bool
        True if the time frame is valid (no missing data), False otherwise.
    """
    feature_col = pair_feature_format.format(
        ASSET1=pair[0], ASSET2=pair[1], FEATURE=feature_name
    )

    if timestamp_col and timestamp_col in df.columns:
        ts = df[timestamp_col]
    else:
        if not isinstance(df.index, pd.DatetimeIndex):
            raise ValueError(
                "DataFrame neither has the timestamp column nor a DatetimeIndex."
            )
        ts = df.index

    mask = (ts >= start) & (ts < end)
    if mask.sum() == 0:
        return False

    data_slice = df.loc[mask, feature_col]
    return not data_slice.isna().any()


pair_identifier = CONFIG["DATA"]["features"]["pair_identifier"]

valid_intervals_per_pair = {}
for pair in asset_pairs:
    valid_intervals = []
    for start, end in intervals:
        if is_timeframe_valid(
            features_df,
            pair,
            start,
            end,
            feature_name=pair_identifier,
            pair_feature_format=pair_feature_format,
            timestamp_col=timestamp_col):

            valid_intervals.append((start, end))
    valid_intervals_per_pair[pair] = valid_intervals
    print(f"Pair {pair} has {len(valid_intervals)} valid intervals out of {len(intervals)} total intervals.")

Pair ('AAVE', 'SUI') has 6 valid intervals out of 363 total intervals.
Pair ('AAVE', 'TRX') has 2 valid intervals out of 363 total intervals.
Pair ('ADA', 'BTC') has 6 valid intervals out of 363 total intervals.
Pair ('ADA', 'DOGE') has 15 valid intervals out of 363 total intervals.
Pair ('ADA', 'HBAR') has 25 valid intervals out of 363 total intervals.
Pair ('ADA', 'LTC') has 7 valid intervals out of 363 total intervals.
Pair ('ADA', 'SUI') has 7 valid intervals out of 363 total intervals.
Pair ('ADA', 'XLM') has 9 valid intervals out of 363 total intervals.
Pair ('ADA', 'XRP') has 11 valid intervals out of 363 total intervals.
Pair ('APT', 'AVAX') has 6 valid intervals out of 363 total intervals.
Pair ('ARB', 'ATOM') has 11 valid intervals out of 363 total intervals.
Pair ('ARB', 'AVAX') has 9 valid intervals out of 363 total intervals.
Pair ('ARB', 'DOT') has 12 valid intervals out of 363 total intervals.
Pair ('ARB', 'ETC') has 20 valid intervals out of 363 total intervals.
Pair ('

## Dataset Structure Overview

**Important**: The dataset has a specific structure:
- **Multiple asset pairs**: Each pair trades independently
- **2-day trading windows**: Each episode is exactly 2 days long
- **Not continuous**: Windows are discrete episodes, not a continuous time series
- **Strategy resets**: At the start of each 2-day window, the strategy resets to neutral position

This structure allows the model to learn from diverse trading scenarios across different pairs and time periods.

## Rule-Based Pairs Trading Strategy

This section implements a simple rule-based statistical arbitrage strategy:
- When z-score crosses above entry threshold: short rich asset, long poor asset
- When z-score crosses below -entry threshold: long rich asset, short poor asset
- When z-score crosses back under exit threshold (from above or below): close all positions

Action -1 (Short Spread): 
When z_score > 1.5, short first asset, long second asset

Action 0 (Neutral):
When |z_score| < 0.5, closed positions, previous action is persisted

Action +1 (Long Spread):
When z_score < -1.5, long first asset, short second asset

In [12]:
class RuleBasedPairsStrategy:
    """
    Rule-based pairs trading strategy for generating expert demonstrations.
    
    Strategy rules:
    - When z-score > entry_threshold: short rich asset, long poor asset (action = -1)
    - When z-score < -entry_threshold: long rich asset, short poor asset (action = +1)
    - When |z-score| < exit_threshold: close all positions (action = 0)
    """
    
    def __init__(self, entry_threshold: float = 1.5, exit_threshold: float = 0.5):
        self.entry_threshold = entry_threshold
        self.exit_threshold = exit_threshold
        self.position = 0  # -1: short spread, 0: neutral, +1: long spread
        
    def get_action(self, z_score: float) -> int:
        """
        Determine trading action based on z-score.
        
        Parameters
        ----------
        z_score : float
            Current z-score of the spread
            
        Returns
        -------
        int
            Action: -1 (short spread), 0 (neutral), +1 (long spread)
        """
        # Entry signals
        if z_score > self.entry_threshold:
            # Spread is too high -> short the spread (expect mean reversion)
            self.position = -1
        elif z_score < -self.entry_threshold:
            # Spread is too low -> long the spread (expect mean reversion)
            self.position = 1
        # Exit signals
        elif abs(z_score) < self.exit_threshold:
            # Spread has reverted to mean -> close position
            self.position = 0
        # else: maintain current position
        
        return self.position
    
    def reset(self):
        """Reset the strategy state."""
        self.position = 0


# Test the strategy
strategy = RuleBasedPairsStrategy(
    entry_threshold=CONFIG["Strategy"]["entry"],
    exit_threshold=CONFIG["Strategy"]["exit"]
)

# Test with sample z-scores
test_z_scores = [0.0, 2.0, 1.8, 0.8, 0.3, 0.7, -0.2, -2.0, -1.5, 0.4, 0.1]
print("Testing rule-based strategy:")
print("Z-score -> Action (Position)")
for z in test_z_scores:
    action = strategy.get_action(z)
    print(f"  {z:5.1f} -> {action:2d}")

Testing rule-based strategy:
Z-score -> Action (Position)
    0.0 ->  0
    2.0 -> -1
    1.8 -> -1
    0.8 -> -1
    0.3 ->  0
    0.7 ->  0
   -0.2 ->  0
   -2.0 ->  1
   -1.5 ->  1
    0.4 ->  0
    0.1 ->  0


## Generate Offline Dataset from Rule-Based Strategy

This section generates an offline dataset by applying the rule-based strategy to historical data. 

**Important**: We store data per pair and per interval, NOT as a combined dataset. During training, we'll iterate through pairs and their intervals.

In [13]:
def generate_interval_dataset(
    df: pd.DataFrame,
    pair: tuple[str, str],
    interval: tuple[pd.Timestamp, pd.Timestamp],
    strategy: RuleBasedPairsStrategy,
    z_score_feature: str = "spreadNorm",
    pair_feature_format: str = "{ASSET1}_{ASSET2}_{FEATURE}",
    single_asset_features: list[str] = None,
    timestamp_col: str = "datetime",
) -> dict:
    """
    Generate dataset for a SINGLE interval (2-day window) for a pair.
    
    Returns
    -------
    dict with keys:
        - observations: np.ndarray of shape (N, obs_dim)
        - actions: np.ndarray of shape (N,)
        - rewards: np.ndarray of shape (N,)
        - timestamps: list of timestamps
    """
    observations = []
    actions = []
    rewards = []
    timestamps_list = []
    
    # Construct column names
    z_score_col = pair_feature_format.format(
        ASSET1=pair[0], ASSET2=pair[1], FEATURE=z_score_feature
    )
    
    # Get single asset feature columns for state representation
    state_cols = []
    if single_asset_features:
        for asset in pair:
            for feat in single_asset_features:
                col = f"{asset}_{feat}"
                if col in df.columns:
                    state_cols.append(col)
    
    # Add z-score to state
    state_cols.append(z_score_col)
    
    # Get data for this interval
    start, end = interval
    if timestamp_col and timestamp_col in df.columns:
        ts = df[timestamp_col]
    else:
        ts = df.index
        
    mask = (ts >= start) & (ts < end)
    interval_df = df[mask].copy()
    
    if interval_df.empty or z_score_col not in interval_df.columns:
        return None
        
    # Check if we have all required columns
    missing_cols = [col for col in state_cols if col not in interval_df.columns]
    if missing_cols:
        return None
    
    # Reset strategy for this interval
    strategy.reset()
    
    # Generate state-action pairs for this interval
    for idx, row in interval_df.iterrows():
        # Get state
        state = row[state_cols].values.astype(np.float32)
        
        # Skip if any NaN in state
        if np.any(np.isnan(state)):
            continue
        
        # Get z-score and determine action
        z_score = row[z_score_col]
        action = strategy.get_action(z_score)
        
        # Simple reward: negative absolute z-score (encourage mean reversion)
        reward = -abs(z_score)
        
        observations.append(state)
        actions.append(action)
        rewards.append(reward)
        
        if timestamp_col and timestamp_col in df.columns:
            timestamps_list.append(row[timestamp_col])
        else:
            timestamps_list.append(idx)
    
    # Convert to numpy arrays
    if len(observations) == 0:
        return None
        
    return {
        "observations": np.array(observations, dtype=np.float32),
        "actions": np.array(actions, dtype=np.int32),
        "rewards": np.array(rewards, dtype=np.float32),
        "timestamps": timestamps_list,
    }


def generate_offline_dataset(
    df: pd.DataFrame,
    pair: tuple[str, str],
    intervals: list[tuple[pd.Timestamp, pd.Timestamp]],
    strategy: RuleBasedPairsStrategy,
    z_score_feature: str = "spreadNorm",
    pair_feature_format: str = "{ASSET1}_{ASSET2}_{FEATURE}",
    single_asset_features: list[str] = None,
    timestamp_col: str = "datetime",
) -> dict:
    """
    Generate offline dataset using rule-based strategy.
    
    Returns
    -------
    dict with keys:
        - observations: np.ndarray of shape (N, obs_dim)
        - actions: np.ndarray of shape (N,)
        - rewards: np.ndarray of shape (N,)
        - terminals: np.ndarray of shape (N,)
        - timestamps: list of timestamps
    """
    
    observations = []
    actions = []
    rewards = []
    terminals = []
    timestamps_list = []
    
    # Construct column names
    z_score_col = pair_feature_format.format(
        ASSET1=pair[0], ASSET2=pair[1], FEATURE=z_score_feature
    )
    
    # Get single asset feature columns for state representation
    state_cols = []
    if single_asset_features:
        for asset in pair:
            for feat in single_asset_features:
                col = f"{asset}_{feat}"
                if col in df.columns:
                    state_cols.append(col)
    
    # Add z-score to state
    state_cols.append(z_score_col)
    
    print(f"\nGenerating dataset for pair {pair}")
    print(f"State features ({len(state_cols)}): {state_cols[:5]}...")
    
    # Process each interval
    for start, end in intervals:
        # Get data for this interval
        if timestamp_col and timestamp_col in df.columns:
            ts = df[timestamp_col]
        else:
            ts = df.index
            
        mask = (ts >= start) & (ts < end)
        interval_df = df[mask].copy()
        
        if interval_df.empty or z_score_col not in interval_df.columns:
            continue
            
        # Check if we have all required columns
        missing_cols = [col for col in state_cols if col not in interval_df.columns]
        if missing_cols:
            continue
        
        # Reset strategy for new interval
        strategy.reset()
        
        # Generate state-action pairs
        for idx, row in interval_df.iterrows():
            # Get state
            state = row[state_cols].values.astype(np.float32)
            
            # Skip if any NaN in state
            if np.any(np.isnan(state)):
                continue
            
            # Get z-score and determine action
            z_score = row[z_score_col]
            action = strategy.get_action(z_score)
            
            # Simple reward: negative absolute z-score (encourage mean reversion)
            reward = -abs(z_score)
            
            observations.append(state)
            actions.append(action)
            rewards.append(reward)
            terminals.append(False)  # Only last step in interval is terminal
            
            if timestamp_col and timestamp_col in df.columns:
                timestamps_list.append(row[timestamp_col])
            else:
                timestamps_list.append(idx)
        
        # Mark last step as terminal
        if len(terminals) > 0:
            terminals[-1] = True
    
    # Convert to numpy arrays
    observations = np.array(observations, dtype=np.float32)
    actions = np.array(actions, dtype=np.int32)
    rewards = np.array(rewards, dtype=np.float32)
    terminals = np.array(terminals, dtype=bool)
    
    print(f"Generated {len(observations)} transitions")
    print(f"  Observations shape: {observations.shape}")
    print(f"  Actions distribution: {np.bincount(actions + 1)}")  # +1 to handle -1,0,1
    print(f"  Rewards range: [{rewards.min():.3f}, {rewards.max():.3f}]")
    
    return {
        "observations": observations,
        "actions": actions,
        "rewards": rewards,
        "terminals": terminals,
        "timestamps": timestamps_list,
    }


def generate_interval_dataset_with_pnl(
    df: pd.DataFrame,
    pair: tuple[str, str],
    interval: tuple[pd.Timestamp, pd.Timestamp],
    strategy: RuleBasedPairsStrategy,
    transaction_cost_bps: float = 3.5,
    z_score_feature: str = "spreadNorm",
    pair_feature_format: str = "{ASSET1}_{ASSET2}_{FEATURE}",
    single_asset_features: list[str] = None,
    timestamp_col: str = "datetime",
) -> dict:
    """
    Generate dataset with PnL-based rewards (portfolio returns - transaction costs).
    
    Reward calculation:
    - Observe state_t, take action_t
    - Reward_t = portfolio return from t to t+1 based on action_t, minus transaction costs
    - Action = -1 (short spread): short asset1, long asset2
    - Action = 1 (long spread): long asset1, short asset2  
    - Action = 0 (neutral): no position, return = 0
    
    Returns
    -------
    dict with keys:
        - observations: np.ndarray of shape (N, obs_dim)
        - actions: np.ndarray of shape (N,)
        - rewards: np.ndarray of shape (N,)
        - timestamps: list of timestamps
    """
    # First pass: collect all states and actions
    states_list = []
    actions_list = []
    prices1_list = []
    prices2_list = []
    timestamps_list = []
    
    # Construct column names
    z_score_col = pair_feature_format.format(
        ASSET1=pair[0], ASSET2=pair[1], FEATURE=z_score_feature
    )
    
    # Get price columns for each asset
    asset1_price_col = f"{pair[0]}_close"
    asset2_price_col = f"{pair[1]}_close"
    
    # Get single asset feature columns for state representation
    state_cols = []
    if single_asset_features:
        for asset in pair:
            for feat in single_asset_features:
                col = f"{asset}_{feat}"
                if col in df.columns:
                    state_cols.append(col)
    
    # Add z-score to state
    state_cols.append(z_score_col)
    
    # Get data for this interval
    start, end = interval
    if timestamp_col and timestamp_col in df.columns:
        ts = df[timestamp_col]
    else:
        ts = df.index
        
    mask = (ts >= start) & (ts < end)
    interval_df = df[mask].copy()
    
    if interval_df.empty or z_score_col not in interval_df.columns:
        return None
        
    # Check if we have all required columns
    missing_cols = [col for col in state_cols if col not in interval_df.columns]
    if missing_cols or asset1_price_col not in interval_df.columns or asset2_price_col not in interval_df.columns:
        return None
    
    # Reset strategy for this interval
    strategy.reset()
    
    # First pass: collect states, actions, and prices
    for idx, row in interval_df.iterrows():
        # Get state
        state = row[state_cols].values.astype(np.float32)
        
        # Skip if any NaN in state
        if np.any(np.isnan(state)):
            continue
        
        # Get current prices
        price1 = row[asset1_price_col]
        price2 = row[asset2_price_col]
        
        if np.isnan(price1) or np.isnan(price2):
            continue
        
        # Get z-score and determine action
        z_score = row[z_score_col]
        action = strategy.get_action(z_score)
        
        states_list.append(state)
        actions_list.append(action)
        prices1_list.append(price1)
        prices2_list.append(price2)
        
        if timestamp_col and timestamp_col in df.columns:
            timestamps_list.append(row[timestamp_col])
        else:
            timestamps_list.append(idx)
    
    if len(states_list) == 0:
        return None
    
    # Second pass: calculate rewards
    # Reward[t] = return from taking action[t] at time t, observing price change to t+1
    rewards_list = []
    prev_action = 0  # Track previous action for transaction costs
    
    for t in range(len(states_list)):
        action_t = actions_list[t]
        
        # Check if we have next prices to calculate return
        if t < len(states_list) - 1:
            # Calculate returns from t to t+1
            price1_t = prices1_list[t]
            price2_t = prices2_list[t]
            price1_next = prices1_list[t + 1]
            price2_next = prices2_list[t + 1]
            
            ret1 = (price1_next - price1_t) / price1_t if price1_t != 0 else 0
            ret2 = (price2_next - price2_t) / price2_t if price2_t != 0 else 0
            
            # Calculate portfolio return based on action taken at t
            if action_t == -1:
                # Short spread: short asset1, long asset2
                portfolio_return = -ret1 + ret2
            elif action_t == 1:
                # Long spread: long asset1, short asset2
                portfolio_return = ret1 - ret2
            else:
                # No position
                portfolio_return = 0.0
            
            # Calculate transaction costs (applied when position changes)
            transaction_cost = 0.0
            if action_t != prev_action:
                # Position change: pay transaction cost on both legs
                transaction_cost = 2 * (transaction_cost_bps / 10000)
            
            # Final reward = portfolio return - transaction costs
            reward = portfolio_return - transaction_cost
        else:
            # Last observation: no next price, reward = 0
            reward = 0.0
        
        rewards_list.append(reward)
        prev_action = action_t
    
    # Convert to numpy arrays
    observations = np.array(states_list, dtype=np.float32)
    actions = np.array(actions_list, dtype=np.int32)
    rewards = np.array(rewards_list, dtype=np.float32)
        
    return {
        "observations": observations,
        "actions": actions,
        "rewards": rewards,
        "timestamps": timestamps_list,
    }


# Store data organized by pair and interval (NOT combined into one dataset)
z_score_feature = CONFIG["Strategy"]["z-score"]
entry_threshold = CONFIG["Strategy"]["entry"]
exit_threshold = CONFIG["Strategy"]["exit"]

# Create strategy
strategy = RuleBasedPairsStrategy(entry_threshold, exit_threshold)

# Store data per pair-interval for iteration during training
pair_interval_data = {}  # {pair: {interval_idx: data}}
total_intervals = 0
pairs_with_data = []

print("\n" + "="*60)
print("Organizing Data by Pair and Interval")
print("="*60)

for pair, valid_ints in valid_intervals_per_pair.items():
    if len(valid_ints) == 0:
        continue
    
    print(f"\nProcessing pair {pair} with {len(valid_ints)} valid 2-day windows...")
    pair_interval_data[pair] = {}
    
    valid_count = 0
    for interval_idx, interval in enumerate(valid_ints):
        # Generate dataset for this specific interval
        interval_data = generate_interval_dataset(
            features_df,
            pair,
            interval,
            strategy,
            z_score_feature=z_score_feature,
            pair_feature_format=pair_feature_format,
            single_asset_features=single_asset_features[:5],  # Use first 5 features
            timestamp_col=timestamp_col,
        )
        
        if interval_data is not None and len(interval_data["observations"]) > 0:
            pair_interval_data[pair][interval_idx] = {
                "data": interval_data,
                "interval": interval,
            }
            valid_count += 1
    
    if valid_count > 0:
        pairs_with_data.append(pair)
        total_intervals += valid_count
        print(f"  ✓ Stored {valid_count} valid intervals for pair {pair}")
    else:
        # Remove pair if no valid data
        del pair_interval_data[pair]

print("\n" + "="*60)
print("Data Organization Summary")
print("="*60)
print(f"Total pairs with data: {len(pairs_with_data)}")
print(f"Total 2-day intervals: {total_intervals}")
print(f"Pairs: {pairs_with_data}")
print("="*60)


Organizing Data by Pair and Interval

Processing pair ('AAVE', 'SUI') with 6 valid 2-day windows...
  ✓ Stored 6 valid intervals for pair ('AAVE', 'SUI')

Processing pair ('AAVE', 'TRX') with 2 valid 2-day windows...
  ✓ Stored 2 valid intervals for pair ('AAVE', 'TRX')

Processing pair ('ADA', 'BTC') with 6 valid 2-day windows...
  ✓ Stored 6 valid intervals for pair ('ADA', 'BTC')

Processing pair ('ADA', 'DOGE') with 15 valid 2-day windows...
  ✓ Stored 15 valid intervals for pair ('ADA', 'DOGE')

Processing pair ('ADA', 'HBAR') with 25 valid 2-day windows...
  ✓ Stored 25 valid intervals for pair ('ADA', 'HBAR')

Processing pair ('ADA', 'LTC') with 7 valid 2-day windows...
  ✓ Stored 7 valid intervals for pair ('ADA', 'LTC')

Processing pair ('ADA', 'SUI') with 7 valid 2-day windows...
  ✓ Stored 7 valid intervals for pair ('ADA', 'SUI')

Processing pair ('ADA', 'XLM') with 9 valid 2-day windows...


KeyboardInterrupt: 

## Dataset Statistics

Let's compute some statistics about our organized data.

## Example Dataset Inspection

Let's look at an example dataset to see what the data and actions look like.

In [ ]:
if len(pair_interval_data) > 0:
    # Get the first pair and first interval
    example_pair = list(pair_interval_data.keys())[0]
    example_interval_idx = list(pair_interval_data[example_pair].keys())[0]
    example_data = pair_interval_data[example_pair][example_interval_idx]["data"]
    example_interval = pair_interval_data[example_pair][example_interval_idx]["interval"]
    
    print(f"\n{'='*80}")
    print(f"Example Dataset from Pair: {example_pair}")
    print(f"Interval: {example_interval[0]} to {example_interval[1]}")
    print(f"{'='*80}\n")
    
    # Create a DataFrame for better visualization
    example_df = pd.DataFrame({
        'timestamp': example_data['timestamps'][:20],
        'observation_0': example_data['observations'][:20, 0],
        'observation_1': example_data['observations'][:20, 1] if example_data['observations'].shape[1] > 1 else None,
        'observation_last': example_data['observations'][:20, -1],  # Last feature (z-score)
        'action': example_data['actions'][:20],
        'reward': example_data['rewards'][:20],
    })
    
    print("First 20 samples from this interval:")
    print(example_df.to_string(index=False))
    
    print(f"\n{'='*80}")
    print("Action Encoding:")
    print("  -1 = Short Spread (z-score too high, expect mean reversion down)")
    print("   0 = Neutral (close positions, z-score near mean)")
    print("  +1 = Long Spread (z-score too low, expect mean reversion up)")
    print(f"{'='*80}\n")
    
    # Show action counts for this example
    action_counts = np.bincount(example_data['actions'] + 1)
    print("Action distribution in this example interval:")
    print(f"  Short (-1): {action_counts[0]} samples")
    print(f"  Neutral (0): {action_counts[1]} samples")
    print(f"  Long (+1): {action_counts[2]} samples")
    print(f"  Total: {len(example_data['actions'])} samples")
    
else:
    print("No data available to display")

### Strategy Design Decisions

**Current Implementation: Discrete Actions**
- Actions: {-1, 0, 1} (Full Short, Neutral, Full Long)
- Simple and interpretable
- Uses DiscreteCQL algorithm

**Alternative: Continuous Actions (Partial Positions)**
To allow fractional positions (e.g., 0.5 = half position, -0.3 = 30% short):

1. **Modify Strategy to Output Continuous Actions:**
   ```python
   def get_continuous_action(self, z_score):
       """Returns action in [-1, 1] based on z-score strength."""
       if z_score > self.entry_threshold:
           # Stronger signal → larger short position
           strength = min((z_score - self.entry_threshold) / self.entry_threshold, 1.0)
           return -strength  # e.g., -0.5 for moderate signal
       elif z_score < -self.entry_threshold:
           strength = min((-z_score - self.entry_threshold) / self.entry_threshold, 1.0)
           return strength   # e.g., 0.7 for strong signal
       else:
           return 0.0  # Neutral
   ```

2. **Switch from DiscreteCQL to Continuous CQL:**
   ```python
   from d3rlpy.algos import CQLConfig  # Continuous version
   
   cql = CQLConfig(
       learning_rate=3e-4,
       batch_size=256,
       actor_learning_rate=3e-4,  # Additional: actor network learning rate
       alpha=5.0,
       # ... other params
   ).create(device=device_str)
   ```

3. **Update Dataset Creation:**
   - Actions are already continuous (no need for +1 conversion)
   - Rewards scale with position size
   
4. **Trade-offs:**
   - ✅ More flexible position sizing
   - ✅ Can express confidence levels
   - ✅ Better risk management
   - ❌ Harder to learn (larger action space)
   - ❌ More hyperparameter tuning needed

For this notebook, we use **discrete actions** for simplicity and faster training. For production, continuous actions are recommended.

In [ ]:
if len(pair_interval_data) > 0:
    # Compute statistics
    total_samples = 0
    all_obs_dims = set()
    all_actions_list = []
    
    for pair, intervals in pair_interval_data.items():
        for interval_idx, interval_info in intervals.items():
            data = interval_info["data"]
            total_samples += len(data["observations"])
            all_obs_dims.add(data["observations"].shape[1])
            all_actions_list.extend(data["actions"])
    
    obs_dim = list(all_obs_dims)[0] if len(all_obs_dims) == 1 else None
    
    print("\nDataset Statistics:")
    print(f"  Total samples across all pair-intervals: {total_samples:,}")
    print(f"  Observation dimension: {obs_dim}")
    print(f"  Action distribution: {dict(zip([-1, 0, 1], np.bincount(np.array(all_actions_list) + 1)))}")
    
    if obs_dim is None:
        print("  ⚠ Warning: Inconsistent observation dimensions across pairs!")
else:
    print("Cannot compute statistics - no data generated")
    obs_dim = None

## Summary

This notebook demonstrates:

1. **Rule-Based Strategy**: Implemented a simple statistical arbitrage strategy that:
   - Goes short on spread when z-score > entry threshold (1.5)
   - Goes long on spread when z-score < -entry threshold (-1.5)
   - Closes positions when |z-score| < exit threshold (0.5)

2. **Data Organization by Pair and Interval**:
   - Data is stored per pair and per 2-day interval (NOT combined into one dataset)
   - Each pair has multiple discrete 2-day trading windows
   - Strategy resets at the beginning of each interval

3. **Training Approach - Iterating over Pairs and Intervals**:
   ```python
   for epoch in epochs:
       for pair, interval in train_pair_intervals:
           # Train on this specific pair-interval
           data = pair_interval_data[pair][interval]
           train_step(model, data)
   ```
   This approach:
   - Treats each pair-interval as a separate training instance
   - Model sees diverse examples from different pairs and time periods
   - More suitable for the discrete, non-continuous nature of the data

4. **Conservative Q-Learning (CQL)**: Train an offline RL agent using observations and actions from our simple strategy:
   - Uses d3rlpy's DiscreteCQL implementation
   - Learns to optimize expected returns with conservative Q-values
   - Trained on diverse 2-day trading scenarios across different pairs

The trained CQL model learns a **general policy** applicable across different pairs and market conditions.

## Extension: Continuous Action Space (Partial Positions)

The current implementation uses discrete actions {-1, 0, 1}. Here's how to extend it to continuous actions for partial position sizing:

In [ ]:
# Example: Modified Strategy with Continuous Actions
# NOTE: This is just an example - it creates a separate strategy instance
# and does NOT replace the main 'strategy' variable used elsewhere
class ContinuousRuleBasedStrategy:
    """
    Statistical arbitrage strategy with continuous position sizing.
    Actions range from -1 (full short) to 1 (full long).
    """
    
    def __init__(self, entry_threshold=1.5, exit_threshold=0.5):
        self.entry_threshold = entry_threshold
        self.exit_threshold = exit_threshold
        self.current_position = 0.0  # Current position size [-1, 1]
    
    def get_action(self, z_score: float) -> float:
        """
        Returns continuous action based on z-score.
        
        Action interpretation:
        - 1.0: Full long position (100% long the spread)
        - 0.5: Half long position (50% long)
        - 0.0: Neutral (no position)
        - -0.5: Half short position (50% short)
        - -1.0: Full short position (100% short)
        """
        # If already in position, check exit condition
        if abs(self.current_position) > 0 and abs(z_score) < self.exit_threshold:
            self.current_position = 0.0
            return 0.0
        
        # Entry logic with position sizing
        if z_score > self.entry_threshold:
            # Short the spread - scale by signal strength
            # Stronger deviation → larger position
            strength = min((z_score - self.entry_threshold) / self.entry_threshold, 1.0)
            action = -strength  # Range: [-1, 0]
            self.current_position = action
            return action
            
        elif z_score < -self.entry_threshold:
            # Long the spread - scale by signal strength
            strength = min((-z_score - self.entry_threshold) / self.entry_threshold, 1.0)
            action = strength  # Range: [0, 1]
            self.current_position = action
            return action
        
        # No strong signal - stay neutral or maintain position
        return self.current_position

# Example usage (using a separate variable name)
print("="*60)
print("Continuous Action Strategy Examples")
print("="*60)

continuous_strategy = ContinuousRuleBasedStrategy(entry_threshold=1.5, exit_threshold=0.5)

test_cases = [
    (2.5, "Strong mean reversion signal"),
    (1.8, "Moderate mean reversion signal"),
    (1.5, "Just at entry threshold"),
    (0.8, "Weak signal - maintain position"),
    (0.3, "Exit signal - close position"),
    (-2.0, "Strong opposite signal"),
]

for z_score, description in test_cases:
    action = continuous_strategy.get_action(z_score)
    position_pct = abs(action) * 100
    direction = "SHORT" if action < 0 else "LONG" if action > 0 else "NEUTRAL"
    
    print(f"\nZ-Score: {z_score:+.2f} | {description}")
    print(f"  → Action: {action:+.3f} ({direction} {position_pct:.1f}%)")
    print(f"  → Current position: {continuous_strategy.current_position:+.3f}")

print("\n" + "="*60)
print("Key Advantages of Continuous Actions:")
print("="*60)
print("✓ Proportional risk-taking (stronger signals → larger positions)")
print("✓ Gradual position adjustments")
print("✓ Better capital utilization")
print("✓ More realistic trading behavior")
print("\nTo use with CQL: Switch from DiscreteCQLConfig to CQLConfig")
print("and remove the +1 action conversion (actions already continuous)")
print("\nNote: This example uses 'continuous_strategy' variable,")
print("not 'strategy', so it won't interfere with the main workflow.")


---

# Conservative Q-Learning (CQL) with d3rlpy

Now we'll train an offline reinforcement learning agent using Conservative Q-Learning (CQL). CQL is designed for offline RL and prevents overestimation of Q-values on out-of-distribution actions.

## Convert Pair-Interval Data to MDPDataset

First, we need to convert our pair-interval data structure into d3rlpy's `MDPDataset` format. We'll create one dataset per pair-interval, marking the last transition in each interval as terminal.

In [ ]:
from d3rlpy.dataset import MDPDataset

def convert_to_mdp_dataset(interval_data: dict) -> MDPDataset:
    """
    Convert interval data to d3rlpy MDPDataset format.
    
    Parameters
    ----------
    interval_data : dict
        Dictionary with keys: observations, actions, rewards, timestamps
        
    Returns
    -------
    MDPDataset
        d3rlpy dataset with terminal flags
    """
    observations = interval_data["observations"]
    actions = interval_data["actions"]
    rewards = interval_data["rewards"]
    
    # Convert actions from {-1, 0, 1} to {0, 1, 2} for d3rlpy
    # d3rlpy expects non-negative action indices
    actions_converted = actions + 1  # -1 -> 0, 0 -> 1, 1 -> 2
    
    # Create terminals array - mark last transition as terminal
    terminals = np.zeros(len(observations), dtype=np.float32)
    if len(terminals) > 0:
        terminals[-1] = 1.0
    
    # Create MDPDataset
    dataset = MDPDataset(
        observations=observations,
        actions=actions_converted,  # Use converted actions
        rewards=rewards,
        terminals=terminals,        # flag the end of episode
    )
    
    return dataset


# Convert all pair-interval data to MDPDatasets
pair_interval_mdp_datasets = {}  # {pair: {interval_idx: MDPDataset}}

print("\n" + "="*60)
print("Converting Pair-Interval Data to MDPDatasets")
print("="*60)

if len(pair_interval_data) > 0:
    for pair, intervals in pair_interval_data.items():
        pair_interval_mdp_datasets[pair] = {}
        
        for interval_idx, interval_info in intervals.items():
            interval_data = interval_info["data"]
            
            # Convert to MDPDataset
            mdp_dataset = convert_to_mdp_dataset(interval_data)
            pair_interval_mdp_datasets[pair][interval_idx] = {
                "dataset": mdp_dataset,
                "interval": interval_info["interval"],
            }
        
        print(f"  ✓ Converted {len(intervals)} intervals for pair {pair}")
    
    print(f"\n{'='*60}")
    print(f"Total pairs with MDPDatasets: {len(pair_interval_mdp_datasets)}")
    print(f"{'='*60}")
else:
    print("No data to convert!")
    pair_interval_mdp_datasets = {}

## Organize CQL Training Data by Time Splits

Organize the MDPDatasets by train/val/test splits (same as BC).

In [ ]:
if len(pair_interval_mdp_datasets) > 0:
    # Get split dates (same as BC)
    train_start_cql = pd.to_datetime(CONFIG["SPLITS"]["train"][0])
    train_end_cql = pd.to_datetime(CONFIG["SPLITS"]["train"][1])
    val_start_cql = pd.to_datetime(CONFIG["SPLITS"]["val"][0])
    val_end_cql = pd.to_datetime(CONFIG["SPLITS"]["val"][1])
    test_start_cql = pd.to_datetime(CONFIG["SPLITS"]["test"][0])
    test_end_cql = pd.to_datetime(CONFIG["SPLITS"]["test"][1])
    
    # Organize pair-intervals by split
    train_pair_intervals_cql = []
    val_pair_intervals_cql = []
    test_pair_intervals_cql = []
    
    for pair, intervals in pair_interval_mdp_datasets.items():
        for interval_idx, interval_info in intervals.items():
            interval_start, interval_end = interval_info["interval"]
            
            # Determine which split this interval belongs to
            if interval_start >= train_start_cql and interval_end <= train_end_cql:
                train_pair_intervals_cql.append((pair, interval_idx))
            elif interval_start >= val_start_cql and interval_end <= val_end_cql:
                val_pair_intervals_cql.append((pair, interval_idx))
            elif interval_start >= test_start_cql and interval_end <= test_end_cql:
                test_pair_intervals_cql.append((pair, interval_idx))
    
    print("\nCQL Data Split by Time:")
    print(f"  Train: {len(train_pair_intervals_cql)} pair-intervals ({train_start_cql.date()} to {train_end_cql.date()})")
    print(f"  Val:   {len(val_pair_intervals_cql)} pair-intervals ({val_start_cql.date()} to {val_end_cql.date()})")
    print(f"  Test:  {len(test_pair_intervals_cql)} pair-intervals ({test_start_cql.date()} to {test_end_cql.date()})")
else:
    print("Cannot split dataset - no data")
    train_pair_intervals_cql = val_pair_intervals_cql = test_pair_intervals_cql = []

## Initialize Discrete CQL Agent

Set up the CQL agent for training on the offline dataset.

In [ ]:
from d3rlpy.algos import DiscreteCQLConfig
from d3rlpy.models import VectorEncoderFactory

if len(train_pair_intervals_cql) > 0 and obs_dim is not None:
    # Get CQL configuration
    cql_config = CONFIG["CQL"]
    
    # Determine device string for d3rlpy (expects "cuda:0", "cpu:0", or boolean)
    if device.type == "cuda":
        device_str = f"cuda:{device.index if device.index is not None else 0}"
    elif device.type == "mps":
        # d3rlpy doesn't support MPS directly, fall back to CPU
        device_str = "cpu:0"
        print("Note: d3rlpy doesn't support MPS, using CPU instead")
    else:
        device_str = "cpu:0"
    
    # Create CQL agent with proper parameters for d3rlpy v2.6.0
    # Note: DiscreteCQL uses 'alpha' for the conservative penalty weight
    cql = DiscreteCQLConfig(
        learning_rate=cql_config["learning_rate"],
        batch_size=cql_config["batch_size"],
        gamma=cql_config["gamma"],
        alpha=cql_config["alpha"],  # This is the conservative weight in DiscreteCQL
        encoder_factory=VectorEncoderFactory(hidden_units=cql_config["q_func_hidden_sizes"]),
    ).create(device=device_str)
    
    print(f"\n{'='*60}")
    print("Discrete CQL Agent Configuration")
    print(f"{'='*60}")
    print(f"Observation dimension: {obs_dim}")
    print("Number of actions: 3 (discrete)")
    print(f"Learning rate: {cql_config['learning_rate']}")
    print(f"Batch size: {cql_config['batch_size']}")
    print(f"Gamma: {cql_config['gamma']}")
    print(f"Alpha (CQL regularization): {cql_config['alpha']}")
    print(f"Conservative weight: {cql_config['conservative_weight']}")
    print(f"Q-function hidden sizes: {cql_config['q_func_hidden_sizes']}")
    print(f"Device: {device}")
    print(f"{'='*60}\n")
else:
    print("Cannot initialize CQL - missing data or observation dimension")
    cql = None

## Train CQL Agent

Train the CQL agent using the observations and actions from our simple strategy.

In [ ]:
from d3rlpy.metrics import TDErrorEvaluator

if cql is not None and len(train_pair_intervals_cql) > 0:
    # Training configuration
    n_steps = CONFIG["CQL"]["n_steps"]
    n_steps_per_epoch = CONFIG["CQL"]["n_steps_per_epoch"]
    n_epochs_cql = n_steps // n_steps_per_epoch
    
    # Setup logging
    cql_logdir = os.path.join(CONFIG["IO"]["tb_logdir"], f"cql_pair_iteration_{len(pairs_with_data)}pairs")
    ensure_dir(cql_logdir)
    
    print(f"\n{'='*60}")
    print("Training CQL Agent")
    print("Training approach: Combined dataset from all intervals")
    print(f"{'='*60}")
    print(f"Total steps: {n_steps:,}")
    print(f"Steps per epoch: {n_steps_per_epoch}")
    print(f"Epochs: {n_epochs_cql}")
    print(f"Train pair-intervals: {len(train_pair_intervals_cql)}")
    print(f"Val pair-intervals: {len(val_pair_intervals_cql)}")
    print(f"{'='*60}\n")
    
    # Combine all training intervals into a single dataset for efficiency
    print("Combining training datasets...")
    all_train_obs = []
    all_train_actions = []
    all_train_rewards = []
    all_train_terminals = []
    
    # Collect raw data from pair_interval_data instead of MDPDatasets
    for pair, interval_idx in train_pair_intervals_cql:
        # Get the original interval data (before conversion to MDPDataset)
        interval_info = pair_interval_data[pair][interval_idx]
        interval_raw = interval_info["data"]
        
        # Get the raw data
        observations = interval_raw["observations"]
        actions = interval_raw["actions"]
        rewards = interval_raw["rewards"]
        
        # Convert actions from {-1, 0, 1} to {0, 1, 2}
        actions_converted = actions + 1
        
        # Create terminals array (mark last as terminal)
        terminals = np.zeros(len(observations), dtype=np.float32)
        if len(terminals) > 0:
            terminals[-1] = 1.0
        
        # Append to lists
        all_train_obs.append(observations)
        all_train_actions.append(actions_converted)
        all_train_rewards.append(rewards)
        all_train_terminals.append(terminals)
    
    # Concatenate all data
    combined_train_obs = np.concatenate(all_train_obs, axis=0)
    combined_train_actions = np.concatenate(all_train_actions, axis=0)
    combined_train_rewards = np.concatenate(all_train_rewards, axis=0)
    combined_train_terminals = np.concatenate(all_train_terminals, axis=0)
    
    # Create combined training dataset
    combined_train_dataset = MDPDataset(
        observations=combined_train_obs,
        actions=combined_train_actions,
        rewards=combined_train_rewards,
        terminals=combined_train_terminals,
    )
    
    print(f"Combined dataset size: {len(combined_train_obs):,} transitions")
    print(f"Action distribution: {np.bincount(combined_train_actions.astype(int))}")
    print()
    
    # Track best model
    best_val_td_error = float('inf')
    models_dir_cql = CONFIG["IO"]["models_dir"]
    ensure_dir(models_dir_cql)
    best_model_path = os.path.join(models_dir_cql, f"cql_best_combined_{len(pairs_with_data)}pairs.d3")
    
    # Training with progress bar
    print("Training CQL agent...")
    results = cql.fit(
        combined_train_dataset,
        n_steps=n_steps,
        n_steps_per_epoch=n_steps_per_epoch,
        show_progress=True,  # Show progress bar
    )
    
    # Validation after training
    print("\nEvaluating on validation set...")
    val_td_errors = []
    
    for pair, interval_idx in val_pair_intervals_cql:
        val_dataset = pair_interval_mdp_datasets[pair][interval_idx]["dataset"]
        
        # Compute TD error as validation metric
        td_error_evaluator = TDErrorEvaluator()
        td_error = td_error_evaluator(cql, val_dataset)
        val_td_errors.append(td_error)
    
    avg_val_td_error = np.mean(val_td_errors) if val_td_errors else float('inf')
    
    print(f"\nValidation TD Error: {avg_val_td_error:.4f}")
    
    # Save model
    cql.save(best_model_path)
    
    print(f"\n{'='*60}")
    print("CQL Training completed!")
    print(f"Validation TD error: {avg_val_td_error:.4f}")
    print(f"Model saved to: {best_model_path}")
    print(f"{'='*60}\n")
else:
    print("Cannot train CQL - missing agent or data")

## Evaluate CQL on Test Set

Evaluate the trained CQL agent on the test set and analyze its performance.

In [ ]:
if cql is not None and len(test_pair_intervals_cql) > 0:
    # Load best model using d3rlpy's load method
    try:
        from d3rlpy.algos import DiscreteCQL
        cql_loaded = DiscreteCQL.from_json(best_model_path)
        cql = cql_loaded  # Replace with loaded model
        print(f"✓ Loaded best CQL model from: {best_model_path}\n")
    except FileNotFoundError:
        print(f"⚠ Model file not found: {best_model_path}")
        print("Using current trained model instead.\n")
    except Exception as e:
        print(f"⚠ Could not load best model: {e}")
        print("Using current trained model instead.\n")
    
    # Evaluate on test set
    print(f"\n{'='*60}")
    print("CQL Test Set Evaluation")
    print(f"{'='*60}")
    
    test_td_errors = []
    all_cql_preds = []
    all_cql_labels = []
    all_cql_rewards = []
    
    for pair, interval_idx in test_pair_intervals_cql:
        # Get the original interval data
        interval_info = pair_interval_data[pair][interval_idx]
        interval_raw = interval_info["data"]
        
        # Get raw data
        observations = interval_raw["observations"]
        actions = interval_raw["actions"]  # Original {-1, 0, 1}
        rewards = interval_raw["rewards"]
        
        # Convert actions for d3rlpy compatibility
        actions_converted = actions + 1  # {0, 1, 2}
        
        # Create terminals
        terminals = np.zeros(len(observations), dtype=np.float32)
        if len(terminals) > 0:
            terminals[-1] = 1.0
        
        # Create temporary dataset for evaluation
        temp_dataset = MDPDataset(
            observations=observations,
            actions=actions_converted,
            rewards=rewards,
            terminals=terminals,
        )
        
        # Get TD error
        td_error_evaluator = TDErrorEvaluator()
        td_error = td_error_evaluator(cql, temp_dataset)
        test_td_errors.append(td_error)
        
        # Predict actions (will be in {0, 1, 2} format)
        predicted_actions = cql.predict(observations)
        
        all_cql_preds.extend(predicted_actions)
        all_cql_labels.extend(actions_converted)  # Use converted for comparison
        all_cql_rewards.extend(rewards)
    
    avg_test_td_error = np.mean(test_td_errors)
    
    print(f"Average Test TD Error: {avg_test_td_error:.4f}")
    print(f"Total test samples: {len(all_cql_labels)}")
    
    # Convert to numpy arrays
    all_cql_preds = np.array(all_cql_preds)
    all_cql_labels = np.array(all_cql_labels)
    all_cql_rewards = np.array(all_cql_rewards)
    
    # Convert actions back from {0, 1, 2} to {-1, 0, 1} for comparison
    all_cql_preds_original = all_cql_preds - 1
    all_cql_labels_original = all_cql_labels - 1
    
    # Action accuracy
    accuracy = (all_cql_preds == all_cql_labels).mean()
    print(f"Action Accuracy: {accuracy:.4f}")
    
    # Action distribution (use unique with return_counts for negative values)
    print("\nAction Distribution:")
    pred_unique, pred_counts = np.unique(all_cql_preds_original, return_counts=True)
    label_unique, label_counts = np.unique(all_cql_labels_original, return_counts=True)
    
    pred_dist = {int(k): int(v) for k, v in zip(pred_unique, pred_counts)}
    label_dist = {int(k): int(v) for k, v in zip(label_unique, label_counts)}
    
    print(f"  Predicted: {pred_dist}")
    print(f"  True:      {label_dist}")
    
    # Confusion matrix
    try:
        from sklearn.metrics import confusion_matrix, classification_report
    except ImportError:
        print("Warning: scikit-learn not installed. Cannot compute confusion matrix.")
        confusion_matrix = None
        classification_report = None
    
    if confusion_matrix is not None:
        cm_cql = confusion_matrix(all_cql_labels_original, all_cql_preds_original, labels=[-1, 0, 1])
        
        print("\nConfusion Matrix:")
        print("             Predicted")
        print("           -1    0    1")
        print(f"True -1 | {cm_cql[0,0]:4d} {cm_cql[0,1]:4d} {cm_cql[0,2]:4d}")
        print(f"      0 | {cm_cql[1,0]:4d} {cm_cql[1,1]:4d} {cm_cql[1,2]:4d}")
        print(f"      1 | {cm_cql[2,0]:4d} {cm_cql[2,1]:4d} {cm_cql[2,2]:4d}")
        
        print("\nClassification Report:")
        print(classification_report(all_cql_labels_original, all_cql_preds_original, 
                                    target_names=['Short Spread (-1)', 'Neutral (0)', 'Long Spread (1)'],
                                    digits=4))
    
    print(f"{'='*60}\n")
else:
    print("Cannot evaluate CQL - missing agent or test data")

## Visualize CQL Results

Visualize the CQL performance with confusion matrix and action distribution.

In [ ]:
if cql is not None and len(test_pair_intervals_cql) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot confusion matrix
    im = axes[0].imshow(cm_cql, cmap='Greens', aspect='auto')
    axes[0].set_xticks([-1, 0, 1])
    axes[0].set_yticks([-1, 0, 1])
    axes[0].set_xticklabels(['Short (-1)', 'Neutral (0)', 'Long (1)'])
    axes[0].set_yticklabels(['Short (-1)', 'Neutral (0)', 'Long (1)'])
    axes[0].set_xlabel('Predicted Action')
    axes[0].set_ylabel('True Action')
    axes[0].set_title('CQL Confusion Matrix')
    
    # Add text annotations
    for i in range(3):
        for j in range(3):
            text = axes[0].text(j-1, i-1, cm_cql[i, j],
                              ha="center", va="center", 
                              color="black" if cm_cql[i, j] < cm_cql.max()/2 else "white",
                              fontsize=14, fontweight='bold')
    
    plt.colorbar(im, ax=axes[0])
    
    # Plot action distribution comparison
    # Use the original space actions {-1, 0, 1} with proper counting
    action_labels = [-1, 0, 1]
    
    # Count actions for each category (ensuring all 3 actions are represented)
    true_counts_cql = np.array([
        np.sum(all_cql_labels_original == -1),
        np.sum(all_cql_labels_original == 0),
        np.sum(all_cql_labels_original == 1)
    ])
    pred_counts_cql = np.array([
        np.sum(all_cql_preds_original == -1),
        np.sum(all_cql_preds_original == 0),
        np.sum(all_cql_preds_original == 1)
    ])
    
    x = np.arange(3)
    width = 0.35
    
    axes[1].bar(x - width/2, true_counts_cql, width, label='True', alpha=0.8, color='steelblue')
    axes[1].bar(x + width/2, pred_counts_cql, width, label='CQL Predicted', alpha=0.8, color='seagreen')
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(['Short (-1)', 'Neutral (0)', 'Long (1)'])
    axes[1].set_ylabel('Count')
    axes[1].set_title('CQL Action Distribution: True vs Predicted')
    axes[1].legend()
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    
    # Save figure
    if CONFIG["EVAL"]["plots"]:
        reports_dir = CONFIG["EVAL"]["reports_dir"]
        ensure_dir(reports_dir)
        fig_path_cql = os.path.join(reports_dir, f"cql_evaluation_pair_iter_{len(pairs_with_data)}pairs.png")
        plt.savefig(fig_path_cql, dpi=150, bbox_inches='tight')
        print(f"\nSaved CQL evaluation plot to: {fig_path_cql}")
    
    plt.show()
else:
    print("Cannot visualize - missing CQL agent or test data")

# Conservative Q-Learning with PnL-Based Rewards

Now we train a second CQL model using realistic portfolio returns minus transaction costs as rewards.

## Generate Dataset with PnL Rewards

Generate a new dataset where rewards are calculated as: **portfolio_return - transaction_costs**

In [ ]:
# Generate PnL-based dataset for CQL
transaction_cost_bps = CONFIG["CQL_PnL"]["transaction_cost_bps"]

print("\n" + "="*60)
print("Generating PnL-Based Dataset for CQL")
print(f"Transaction cost: {transaction_cost_bps} bps ({transaction_cost_bps/100:.4f}%)")
print("="*60)

pair_interval_data_pnl = {}
total_intervals_pnl = 0
pairs_with_data_pnl = []

for pair, valid_ints in valid_intervals_per_pair.items():
    if len(valid_ints) == 0:
        continue
    
    print(f"\nProcessing pair {pair} with {len(valid_ints)} valid intervals...")
    pair_interval_data_pnl[pair] = {}
    
    valid_count = 0
    for interval_idx, interval in enumerate(valid_ints):
        # Generate dataset with PnL rewards
        interval_data = generate_interval_dataset_with_pnl(
            features_df,
            pair,
            interval,
            strategy,
            transaction_cost_bps=transaction_cost_bps,
            z_score_feature=z_score_feature,
            pair_feature_format=pair_feature_format,
            single_asset_features=single_asset_features[:5],
            timestamp_col=timestamp_col,
        )
        
        if interval_data is not None and len(interval_data["observations"]) > 0:
            pair_interval_data_pnl[pair][interval_idx] = {
                "data": interval_data,
                "interval": interval,
            }
            valid_count += 1
    
    if valid_count > 0:
        pairs_with_data_pnl.append(pair)
        total_intervals_pnl += valid_count
        print(f"  ✓ Stored {valid_count} valid intervals for pair {pair}")
    else:
        del pair_interval_data_pnl[pair]

print("\n" + "="*60)
print("PnL Dataset Summary")
print("="*60)
print(f"Total pairs with data: {len(pairs_with_data_pnl)}")
print(f"Total intervals: {total_intervals_pnl}")
print(f"Pairs: {pairs_with_data_pnl}")

# Compute reward statistics
all_rewards_pnl = []
for pair, intervals in pair_interval_data_pnl.items():
    for interval_idx, interval_info in intervals.items():
        all_rewards_pnl.extend(interval_info["data"]["rewards"])

if len(all_rewards_pnl) > 0:
    all_rewards_pnl = np.array(all_rewards_pnl)
    print(f"\nReward Statistics (PnL-based):")
    print(f"  Mean: {all_rewards_pnl.mean():.6f}")
    print(f"  Std: {all_rewards_pnl.std():.6f}")
    print(f"  Min: {all_rewards_pnl.min():.6f}")
    print(f"  Max: {all_rewards_pnl.max():.6f}")
    print(f"  Positive rewards: {(all_rewards_pnl > 0).sum()} ({(all_rewards_pnl > 0).mean()*100:.1f}%)")
    print(f"  Negative rewards: {(all_rewards_pnl < 0).sum()} ({(all_rewards_pnl < 0).mean()*100:.1f}%)")
print("="*60)

## Organize PnL Data by Splits

Split the PnL dataset into train/val/test using the same time-based splits.

In [ ]:
if len(pair_interval_data_pnl) > 0:
    # Get split dates (same as before)
    train_start_pnl = pd.to_datetime(CONFIG["SPLITS"]["train"][0])
    train_end_pnl = pd.to_datetime(CONFIG["SPLITS"]["train"][1])
    val_start_pnl = pd.to_datetime(CONFIG["SPLITS"]["val"][0])
    val_end_pnl = pd.to_datetime(CONFIG["SPLITS"]["val"][1])
    test_start_pnl = pd.to_datetime(CONFIG["SPLITS"]["test"][0])
    test_end_pnl = pd.to_datetime(CONFIG["SPLITS"]["test"][1])
    
    # Organize pair-intervals by split
    train_pair_intervals_pnl = []
    val_pair_intervals_pnl = []
    test_pair_intervals_pnl = []
    
    for pair, intervals in pair_interval_data_pnl.items():
        for interval_idx, interval_info in intervals.items():
            interval_start, interval_end = interval_info["interval"]
            
            # Determine which split this interval belongs to
            if interval_start >= train_start_pnl and interval_end <= train_end_pnl:
                train_pair_intervals_pnl.append((pair, interval_idx))
            elif interval_start >= val_start_pnl and interval_end <= val_end_pnl:
                val_pair_intervals_pnl.append((pair, interval_idx))
            elif interval_start >= test_start_pnl and interval_end <= test_end_pnl:
                test_pair_intervals_pnl.append((pair, interval_idx))
    
    print("\nPnL Data Split by Time:")
    print(f"  Train: {len(train_pair_intervals_pnl)} pair-intervals")
    print(f"  Val:   {len(val_pair_intervals_pnl)} pair-intervals")
    print(f"  Test:  {len(test_pair_intervals_pnl)} pair-intervals")
else:
    print("Cannot split dataset - no PnL data")
    train_pair_intervals_pnl = val_pair_intervals_pnl = test_pair_intervals_pnl = []

## Initialize CQL-PnL Agent

Create the second CQL agent with PnL-based rewards.

In [ ]:
from d3rlpy.algos import DiscreteCQLConfig
from d3rlpy.models import VectorEncoderFactory

if len(train_pair_intervals_pnl) > 0 and obs_dim is not None:
    # Get CQL-PnL configuration
    cql_pnl_config = CONFIG["CQL_PnL"]
    
    # Determine device string for d3rlpy
    if device.type == "cuda":
        device_str = f"cuda:{device.index if device.index is not None else 0}"
    elif device.type == "mps":
        device_str = "cpu:0"
        print("Note: d3rlpy doesn't support MPS, using CPU instead")
    else:
        device_str = "cpu:0"
    
    # Create CQL-PnL agent
    cql_pnl = DiscreteCQLConfig(
        learning_rate=cql_pnl_config["learning_rate"],
        batch_size=cql_pnl_config["batch_size"],
        gamma=cql_pnl_config["gamma"],
        alpha=cql_pnl_config["alpha"],
        encoder_factory=VectorEncoderFactory(hidden_units=cql_pnl_config["q_func_hidden_sizes"]),
    ).create(device=device_str)
    
    print(f"\n{'='*60}")
    print("CQL-PnL Agent Configuration")
    print(f"{'='*60}")
    print(f"Observation dimension: {obs_dim}")
    print("Number of actions: 3 (discrete)")
    print(f"Learning rate: {cql_pnl_config['learning_rate']}")
    print(f"Batch size: {cql_pnl_config['batch_size']}")
    print(f"Gamma: {cql_pnl_config['gamma']}")
    print(f"Alpha (CQL regularization): {cql_pnl_config['alpha']}")
    print(f"Q-function hidden sizes: {cql_pnl_config['q_func_hidden_sizes']}")
    print(f"Device: {device_str}")
    print(f"Transaction cost: {cql_pnl_config['transaction_cost_bps']} bps")
    print(f"{'='*60}\n")
else:
    print("Cannot create CQL-PnL agent - missing training data or observation dimension")
    cql_pnl = None

## Train CQL-PnL Agent

Train the CQL agent on the PnL-based dataset.

In [ ]:
from d3rlpy.metrics import TDErrorEvaluator

if cql_pnl is not None and len(train_pair_intervals_pnl) > 0:
    # Training configuration
    n_steps_pnl = CONFIG["CQL_PnL"]["n_steps"]
    n_steps_per_epoch_pnl = CONFIG["CQL_PnL"]["n_steps_per_epoch"]
    n_epochs_pnl = n_steps_pnl // n_steps_per_epoch_pnl
    
    print(f"\n{'='*60}")
    print("Training CQL-PnL Agent")
    print("Training approach: Combined dataset with PnL rewards")
    print(f"{'='*60}")
    print(f"Total steps: {n_steps_pnl:,}")
    print(f"Steps per epoch: {n_steps_per_epoch_pnl}")
    print(f"Epochs: {n_epochs_pnl}")
    print(f"Train pair-intervals: {len(train_pair_intervals_pnl)}")
    print(f"Val pair-intervals: {len(val_pair_intervals_pnl)}")
    print(f"{'='*60}\n")
    
    # Combine all training intervals into a single dataset
    print("Combining training datasets...")
    all_train_obs_pnl = []
    all_train_actions_pnl = []
    all_train_rewards_pnl = []
    all_train_terminals_pnl = []
    
    for pair, interval_idx in train_pair_intervals_pnl:
        # Get the original interval data
        interval_info = pair_interval_data_pnl[pair][interval_idx]
        interval_raw = interval_info["data"]
        
        # Get the raw data
        observations = interval_raw["observations"]
        actions = interval_raw["actions"]
        rewards = interval_raw["rewards"]
        
        # Convert actions from {-1, 0, 1} to {0, 1, 2}
        actions_converted = actions + 1
        
        # Create terminals array (mark last as terminal)
        terminals = np.zeros(len(observations), dtype=np.float32)
        if len(terminals) > 0:
            terminals[-1] = 1.0
        
        # Append to lists
        all_train_obs_pnl.append(observations)
        all_train_actions_pnl.append(actions_converted)
        all_train_rewards_pnl.append(rewards)
        all_train_terminals_pnl.append(terminals)
    
    # Concatenate all data
    combined_train_obs_pnl = np.concatenate(all_train_obs_pnl, axis=0)
    combined_train_actions_pnl = np.concatenate(all_train_actions_pnl, axis=0)
    combined_train_rewards_pnl = np.concatenate(all_train_rewards_pnl, axis=0)
    combined_train_terminals_pnl = np.concatenate(all_train_terminals_pnl, axis=0)
    
    # Create combined training dataset
    combined_train_dataset_pnl = MDPDataset(
        observations=combined_train_obs_pnl,
        actions=combined_train_actions_pnl,
        rewards=combined_train_rewards_pnl,
        terminals=combined_train_terminals_pnl,
    )
    
    print(f"Combined dataset size: {len(combined_train_obs_pnl):,} transitions")
    print(f"Action distribution: {np.bincount(combined_train_actions_pnl.astype(int))}")
    print(f"Reward stats: mean={combined_train_rewards_pnl.mean():.6f}, std={combined_train_rewards_pnl.std():.6f}")
    print()
    
    # Track best model
    best_val_td_error_pnl = float('inf')
    models_dir_pnl = CONFIG["IO"]["models_dir"]
    ensure_dir(models_dir_pnl)
    best_model_path_pnl = os.path.join(models_dir_pnl, f"cql_pnl_best_{len(pairs_with_data_pnl)}pairs.d3")
    
    # Training with progress bar
    print("Training CQL-PnL agent...")
    results_pnl = cql_pnl.fit(
        combined_train_dataset_pnl,
        n_steps=n_steps_pnl,
        n_steps_per_epoch=n_steps_per_epoch_pnl,
        show_progress=True,
    )
    
    # Validation after training
    print("\nEvaluating on validation set...")
    val_td_errors_pnl = []
    
    for pair, interval_idx in val_pair_intervals_pnl:
        # Get validation data
        interval_info = pair_interval_data_pnl[pair][interval_idx]
        interval_raw = interval_info["data"]
        
        observations = interval_raw["observations"]
        actions = interval_raw["actions"]
        rewards = interval_raw["rewards"]
        actions_converted = actions + 1
        
        terminals = np.zeros(len(observations), dtype=np.float32)
        if len(terminals) > 0:
            terminals[-1] = 1.0
        
        val_dataset = MDPDataset(
            observations=observations,
            actions=actions_converted,
            rewards=rewards,
            terminals=terminals,
        )
        
        # Compute TD error
        td_error_evaluator = TDErrorEvaluator()
        td_error = td_error_evaluator(cql_pnl, val_dataset)
        val_td_errors_pnl.append(td_error)
    
    avg_val_td_error_pnl = np.mean(val_td_errors_pnl) if val_td_errors_pnl else float('inf')
    
    print(f"\nValidation TD Error: {avg_val_td_error_pnl:.4f}")
    
    # Save model
    cql_pnl.save(best_model_path_pnl)
    
    print(f"\n{'='*60}")
    print("CQL-PnL Training completed!")
    print(f"Validation TD error: {avg_val_td_error_pnl:.4f}")
    print(f"Model saved to: {best_model_path_pnl}")
    print(f"{'='*60}\n")
else:
    print("Cannot train CQL-PnL - missing agent or data")

## Evaluate CQL-PnL on Test Set

Evaluate the PnL-optimized CQL agent on the test set.

In [ ]:
if cql_pnl is not None and len(test_pair_intervals_pnl) > 0:
    print(f"\n{'='*60}")
    print("CQL-PnL Test Set Evaluation")
    print(f"{'='*60}")
    
    test_td_errors_pnl = []
    all_pnl_preds = []
    all_pnl_labels = []
    all_pnl_rewards = []
    
    for pair, interval_idx in test_pair_intervals_pnl:
        # Get test data
        interval_info = pair_interval_data_pnl[pair][interval_idx]
        interval_raw = interval_info["data"]
        
        observations = interval_raw["observations"]
        actions = interval_raw["actions"]
        rewards = interval_raw["rewards"]
        actions_converted = actions + 1
        
        terminals = np.zeros(len(observations), dtype=np.float32)
        if len(terminals) > 0:
            terminals[-1] = 1.0
        
        temp_dataset = MDPDataset(
            observations=observations,
            actions=actions_converted,
            rewards=rewards,
            terminals=terminals,
        )
        
        # Get TD error
        td_error_evaluator = TDErrorEvaluator()
        td_error = td_error_evaluator(cql_pnl, temp_dataset)
        test_td_errors_pnl.append(td_error)
        
        # Predict actions
        predicted_actions = cql_pnl.predict(observations)
        
        all_pnl_preds.extend(predicted_actions)
        all_pnl_labels.extend(actions_converted)
        all_pnl_rewards.extend(rewards)
    
    avg_test_td_error_pnl = np.mean(test_td_errors_pnl)
    
    print(f"Average Test TD Error: {avg_test_td_error_pnl:.4f}")
    print(f"Total test samples: {len(all_pnl_labels)}")
    
    # Convert to numpy arrays
    all_pnl_preds = np.array(all_pnl_preds)
    all_pnl_labels = np.array(all_pnl_labels)
    all_pnl_rewards = np.array(all_pnl_rewards)
    
    # Convert actions back to original space
    all_pnl_preds_original = all_pnl_preds - 1
    all_pnl_labels_original = all_pnl_labels - 1
    
    # Action accuracy
    accuracy_pnl = (all_pnl_preds == all_pnl_labels).mean()
    print(f"Action Accuracy: {accuracy_pnl:.4f}")
    
    # Action distribution
    print("\nAction Distribution:")
    pred_unique_pnl, pred_counts_pnl = np.unique(all_pnl_preds_original, return_counts=True)
    label_unique_pnl, label_counts_pnl = np.unique(all_pnl_labels_original, return_counts=True)
    
    pred_dist_pnl = {int(k): int(v) for k, v in zip(pred_unique_pnl, pred_counts_pnl)}
    label_dist_pnl = {int(k): int(v) for k, v in zip(label_unique_pnl, label_counts_pnl)}
    
    print(f"  Predicted: {pred_dist_pnl}")
    print(f"  True:      {label_dist_pnl}")
    
    # Reward statistics
    print(f"\nReward Statistics on Test Set:")
    print(f"  Mean: {all_pnl_rewards.mean():.6f}")
    print(f"  Cumulative: {all_pnl_rewards.sum():.6f}")
    print(f"  Positive: {(all_pnl_rewards > 0).sum()} ({(all_pnl_rewards > 0).mean()*100:.1f}%)")
    
    # Confusion matrix
    try:
        from sklearn.metrics import confusion_matrix, classification_report
    except ImportError:
        print("Warning: scikit-learn not installed. Cannot compute confusion matrix.")
        confusion_matrix = None
        classification_report = None
    
    if confusion_matrix is not None:
        cm_pnl = confusion_matrix(all_pnl_labels_original, all_pnl_preds_original, labels=[-1, 0, 1])
        
        print("\nConfusion Matrix:")
        print("             Predicted")
        print("           -1    0    1")
        print(f"True -1 | {cm_pnl[0,0]:4d} {cm_pnl[0,1]:4d} {cm_pnl[0,2]:4d}")
        print(f"      0 | {cm_pnl[1,0]:4d} {cm_pnl[1,1]:4d} {cm_pnl[1,2]:4d}")
        print(f"      1 | {cm_pnl[2,0]:4d} {cm_pnl[2,1]:4d} {cm_pnl[2,2]:4d}")
        
        print("\nClassification Report:")
        print(classification_report(all_pnl_labels_original, all_pnl_preds_original,
                                    target_names=['Short Spread (-1)', 'Neutral (0)', 'Long Spread (1)'],
                                    digits=4))
    
    print(f"{'='*60}\n")
else:
    print("Cannot evaluate CQL-PnL - missing agent or test data")

## Compare All Models: BC vs CQL vs CQL-PnL

Compare all three models side by side.

# Portfolio Backtesting

Backtest all strategies/models on the test set and plot portfolio value over time.

## Backtest Function

Create a function to simulate portfolio performance with realistic PnL calculation.

In [ ]:
def backtest_strategy(
    model,
    model_type: str,
    pair_interval_data: dict,
    test_intervals: list,
    transaction_cost_bps: float = 3.5,
    initial_capital: float = 10000.0,
) -> dict:
    """
    Backtest a strategy on test data with realistic PnL calculation.
    
    Parameters
    ----------
    model : model object
        BC network, CQL agent, or rule-based strategy
    model_type : str
        'BC', 'CQL', 'CQL_PnL', or 'RuleBased'
    pair_interval_data : dict
        Dictionary with pair-interval data
    test_intervals : list
        List of (pair, interval_idx) tuples for testing
    transaction_cost_bps : float
        Transaction cost in basis points
    initial_capital : float
        Starting portfolio value
        
    Returns
    -------
    dict with keys:
        - portfolio_values: list of portfolio values over time
        - timestamps: list of timestamps
        - actions: list of actions taken
        - returns: list of returns
        - cumulative_return: final cumulative return
    """
    portfolio_value = initial_capital
    portfolio_values = [portfolio_value]
    all_timestamps = []
    all_actions = []
    all_returns = []
    
    prev_action = 0  # Track previous action for transaction costs
    
    # Process each test interval
    for pair, interval_idx in test_intervals:
        interval_info = pair_interval_data[pair][interval_idx]
        interval_raw = interval_info["data"]
        
        observations = interval_raw["observations"]
        timestamps = interval_raw["timestamps"]
        
        # Get prices (assume they're in the dataframe)
        # We need to reconstruct prices from the original data
        interval_start, interval_end = interval_info["interval"]
        
        # Get interval data from features_df
        if "datetime" in features_df.columns:
            ts = features_df["datetime"]
        else:
            ts = features_df.index
        
        mask = (ts >= interval_start) & (ts < interval_end)
        interval_df = features_df[mask].copy()
        
        if interval_df.empty:
            continue
        
        # Get price columns
        asset1_price_col = f"{pair[0]}_close"
        asset2_price_col = f"{pair[1]}_close"
        
        if asset1_price_col not in interval_df.columns or asset2_price_col not in interval_df.columns:
            continue
        
        prices1 = interval_df[asset1_price_col].values
        prices2 = interval_df[asset2_price_col].values
        
        # Align observations with prices - ensure we have enough data
        # We need at least 2 price points to calculate returns
        if len(prices1) < 2 or len(prices2) < 2 or len(observations) == 0:
            continue
        
        # Ensure we don't go out of bounds
        n_obs = min(len(observations), len(prices1) - 1, len(prices2) - 1)
        
        for t in range(n_obs):
            obs_t = observations[t:t+1]  # Shape (1, obs_dim)
            
            # Get action from model
            if model_type == 'BC':
                obs_tensor = torch.FloatTensor(obs_t).to(device)
                with torch.no_grad():
                    logits = model(obs_tensor)
                    action_idx = torch.argmax(logits, dim=1).item()
                action = action_idx - 1  # Convert from {0,1,2} to {-1,0,1}
            
            elif model_type in ['CQL', 'CQL_PnL']:
                # CQL expects actions in {0,1,2} format internally
                action_pred = model.predict(obs_t)[0]  # Returns action in {0,1,2}
                action = action_pred - 1  # Convert to {-1,0,1}
            
            elif model_type == 'RuleBased':
                # Use the rule-based strategy directly
                # Extract z-score from observation (last feature)
                z_score = obs_t[0, -1]
                action = model.get_action(z_score)
            
            else:
                raise ValueError(f"Unknown model type: {model_type}")
            
            # Calculate return from t to t+1
            price1_t = prices1[t]
            price2_t = prices2[t]
            price1_next = prices1[t + 1]
            price2_next = prices2[t + 1]
            
            # Skip if prices are invalid or NaN
            if (price1_t <= 0 or price2_t <= 0 or price1_next <= 0 or price2_next <= 0 or
                np.isnan(price1_t) or np.isnan(price2_t) or 
                np.isnan(price1_next) or np.isnan(price2_next)):
                continue
            
            ret1 = (price1_next - price1_t) / price1_t
            ret2 = (price2_next - price2_t) / price2_t
            
            # Skip if returns are NaN or infinite
            if np.isnan(ret1) or np.isnan(ret2) or np.isinf(ret1) or np.isinf(ret2):
                continue
            
            # Clip extreme returns to prevent bankruptcy (e.g., ±50% max per trade)
            ret1 = np.clip(ret1, -0.5, 0.5)
            ret2 = np.clip(ret2, -0.5, 0.5)
            
            # Calculate portfolio return based on action
            if action == -1:
                # Short spread: short asset1, long asset2
                portfolio_return = -ret1 + ret2
            elif action == 1:
                # Long spread: long asset1, short asset2
                portfolio_return = ret1 - ret2
            else:
                # No position
                portfolio_return = 0.0
            
            # Apply transaction costs if position changes
            transaction_cost = 0.0
            if action != prev_action:
                transaction_cost = 2 * (transaction_cost_bps / 10000)
            
            # Net return after transaction costs
            net_return = portfolio_return - transaction_cost
            
            # Clip net return to prevent portfolio value going negative
            # Maximum loss per trade should not exceed 95% of portfolio
            net_return = max(net_return, -0.95)
            
            # Update portfolio value
            portfolio_value = portfolio_value * (1 + net_return)
            
            # Safety check: if portfolio value is too low, stop trading
            if portfolio_value < initial_capital * 0.01:  # Less than 1% of initial capital
                print(f"Warning: Portfolio value dropped below 1% of initial capital. Stopping backtest.")
                break
            
            # Record
            portfolio_values.append(portfolio_value)
            all_timestamps.append(timestamps[t] if t < len(timestamps) else None)
            all_actions.append(action)
            all_returns.append(net_return)
            
            prev_action = action
    
    cumulative_return = (portfolio_value - initial_capital) / initial_capital
    
    return {
        'portfolio_values': portfolio_values,
        'timestamps': all_timestamps,
        'actions': all_actions,
        'returns': all_returns,
        'cumulative_return': cumulative_return,
        'final_value': portfolio_value,
    }

print("Backtest function defined successfully!")

## Run Backtests

Backtest all models on the test set.

In [ ]:
transaction_cost_bps = CONFIG["CQL_PnL"]["transaction_cost_bps"]
initial_capital = 10000.0

print(f"\n{'='*80}")
print("Running Backtests on Test Set")
print(f"{'='*80}")
print(f"Initial capital: ${initial_capital:,.2f}")
print(f"Transaction cost: {transaction_cost_bps} bps ({transaction_cost_bps/100:.4f}%)")
print(f"Test intervals: {len(test_pair_intervals)}")
print(f"{'='*80}\n")

# Backtest Rule-Based Strategy
if len(test_pair_intervals) > 0:
    print("Backtesting Rule-Based Strategy...")
    strategy_reset = RuleBasedPairsStrategy(entry_threshold, exit_threshold)
    results_rulebased = backtest_strategy(
        strategy_reset,
        'RuleBased',
        pair_interval_data,
        test_pair_intervals,
        transaction_cost_bps=transaction_cost_bps,
        initial_capital=initial_capital,
    )
    print(f"  ✓ Final Value: ${results_rulebased['final_value']:,.2f}")
    print(f"  ✓ Return: {results_rulebased['cumulative_return']*100:.2f}%\n")
else:
    results_rulebased = None
    print("No test data for Rule-Based Strategy\n")

# Backtest BC
if bc_net is not None and len(test_pair_intervals) > 0:
    print("Backtesting Behavior Cloning...")
    bc_net.eval()
    results_bc = backtest_strategy(
        bc_net,
        'BC',
        pair_interval_data,
        test_pair_intervals,
        transaction_cost_bps=transaction_cost_bps,
        initial_capital=initial_capital,
    )
    print(f"  ✓ Final Value: ${results_bc['final_value']:,.2f}")
    print(f"  ✓ Return: {results_bc['cumulative_return']*100:.2f}%\n")
else:
    results_bc = None
    print("No BC model or test data\n")

# Backtest CQL (Z-Score)
if cql is not None and len(test_pair_intervals_cql) > 0:
    print("Backtesting CQL (Z-Score Reward)...")
    results_cql = backtest_strategy(
        cql,
        'CQL',
        pair_interval_data,  # Use original data for prices
        test_pair_intervals_cql,
        transaction_cost_bps=transaction_cost_bps,
        initial_capital=initial_capital,
    )
    print(f"  ✓ Final Value: ${results_cql['final_value']:,.2f}")
    print(f"  ✓ Return: {results_cql['cumulative_return']*100:.2f}%\n")
else:
    results_cql = None
    print("No CQL model or test data\n")

# Backtest CQL-PnL
if cql_pnl is not None and len(test_pair_intervals_pnl) > 0:
    print("Backtesting CQL (PnL Reward)...")
    results_cql_pnl = backtest_strategy(
        cql_pnl,
        'CQL_PnL',
        pair_interval_data_pnl,
        test_pair_intervals_pnl,
        transaction_cost_bps=transaction_cost_bps,
        initial_capital=initial_capital,
    )
    print(f"  ✓ Final Value: ${results_cql_pnl['final_value']:,.2f}")
    print(f"  ✓ Return: {results_cql_pnl['cumulative_return']*100:.2f}%\n")
else:
    results_cql_pnl = None
    print("No CQL-PnL model or test data\n")

print(f"{'='*80}")
print("Backtest Summary")
print(f"{'='*80}")

summary_data = []
if results_rulebased:
    summary_data.append(['Rule-Based', results_rulebased['final_value'], results_rulebased['cumulative_return']*100])
if results_bc:
    summary_data.append(['BC', results_bc['final_value'], results_bc['cumulative_return']*100])
if results_cql:
    summary_data.append(['CQL (Z-Score)', results_cql['final_value'], results_cql['cumulative_return']*100])
if results_cql_pnl:
    summary_data.append(['CQL (PnL)', results_cql_pnl['final_value'], results_cql_pnl['cumulative_return']*100])

if summary_data:
    print(f"{'Strategy':<20} {'Final Value':>15} {'Return (%)':>12}")
    print("-" * 50)
    for row in summary_data:
        print(f"{row[0]:<20} ${row[1]:>14,.2f} {row[2]:>11.2f}%")
print(f"{'='*80}\n")

## Plot Portfolio Value Over Time

Visualize portfolio performance for all strategies.

In [ ]:
if any([results_rulebased, results_bc, results_cql, results_cql_pnl]):
    fig, axes = plt.subplots(2, 1, figsize=(16, 10))
    
    # Plot 1: Portfolio Value Over Time
    ax1 = axes[0]
    
    if results_rulebased:
        steps = range(len(results_rulebased['portfolio_values']))
        ax1.plot(steps, results_rulebased['portfolio_values'], 
                label=f"Rule-Based (Return: {results_rulebased['cumulative_return']*100:.2f}%)",
                linewidth=2, alpha=0.8)
    
    if results_bc:
        steps = range(len(results_bc['portfolio_values']))
        ax1.plot(steps, results_bc['portfolio_values'], 
                label=f"BC (Return: {results_bc['cumulative_return']*100:.2f}%)",
                linewidth=2, alpha=0.8)
    
    if results_cql:
        steps = range(len(results_cql['portfolio_values']))
        ax1.plot(steps, results_cql['portfolio_values'], 
                label=f"CQL Z-Score (Return: {results_cql['cumulative_return']*100:.2f}%)",
                linewidth=2, alpha=0.8)
    
    if results_cql_pnl:
        steps = range(len(results_cql_pnl['portfolio_values']))
        ax1.plot(steps, results_cql_pnl['portfolio_values'], 
                label=f"CQL PnL (Return: {results_cql_pnl['cumulative_return']*100:.2f}%)",
                linewidth=2, alpha=0.8)
    
    # Add initial capital line
    ax1.axhline(y=initial_capital, color='gray', linestyle='--', linewidth=1, alpha=0.5, label='Initial Capital')
    
    ax1.set_xlabel('Time Steps', fontsize=12)
    ax1.set_ylabel('Portfolio Value ($)', fontsize=12)
    ax1.set_title('Portfolio Value Over Time - All Strategies', fontsize=14, fontweight='bold')
    ax1.legend(loc='best', fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # Format y-axis as currency
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    
    # Plot 2: Cumulative Returns (%)
    ax2 = axes[1]
    
    if results_rulebased:
        portfolio_vals = np.array(results_rulebased['portfolio_values'])
        cumulative_returns = (portfolio_vals / initial_capital - 1) * 100
        steps = range(len(cumulative_returns))
        ax2.plot(steps, cumulative_returns, 
                label=f"Rule-Based (Final: {results_rulebased['cumulative_return']*100:.2f}%)",
                linewidth=2, alpha=0.8)
    
    if results_bc:
        portfolio_vals = np.array(results_bc['portfolio_values'])
        cumulative_returns = (portfolio_vals / initial_capital - 1) * 100
        steps = range(len(cumulative_returns))
        ax2.plot(steps, cumulative_returns, 
                label=f"BC (Final: {results_bc['cumulative_return']*100:.2f}%)",
                linewidth=2, alpha=0.8)
    
    if results_cql:
        portfolio_vals = np.array(results_cql['portfolio_values'])
        cumulative_returns = (portfolio_vals / initial_capital - 1) * 100
        steps = range(len(cumulative_returns))
        ax2.plot(steps, cumulative_returns, 
                label=f"CQL Z-Score (Final: {results_cql['cumulative_return']*100:.2f}%)",
                linewidth=2, alpha=0.8)
    
    if results_cql_pnl:
        portfolio_vals = np.array(results_cql_pnl['portfolio_values'])
        cumulative_returns = (portfolio_vals / initial_capital - 1) * 100
        steps = range(len(cumulative_returns))
        ax2.plot(steps, cumulative_returns, 
                label=f"CQL PnL (Final: {results_cql_pnl['cumulative_return']*100:.2f}%)",
                linewidth=2, alpha=0.8)
    
    # Add zero line
    ax2.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5, label='Break Even')
    
    ax2.set_xlabel('Time Steps', fontsize=12)
    ax2.set_ylabel('Cumulative Return (%)', fontsize=12)
    ax2.set_title('Cumulative Returns Over Time - All Strategies', fontsize=14, fontweight='bold')
    ax2.legend(loc='best', fontsize=10)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save figure
    if CONFIG["EVAL"]["plots"]:
        reports_dir = CONFIG["EVAL"]["reports_dir"]
        ensure_dir(reports_dir)
        backtest_path = os.path.join(reports_dir, f"portfolio_backtest_{len(pairs_with_data)}pairs.png")
        plt.savefig(backtest_path, dpi=150, bbox_inches='tight')
        print(f"\nSaved backtest plot to: {backtest_path}")
    
    plt.show()
else:
    print("No backtest results to plot")

## Performance Metrics

Calculate detailed performance metrics for all strategies.

In [ ]:
def calculate_performance_metrics(results: dict, strategy_name: str) -> dict:
    """
    Calculate detailed performance metrics for a strategy.
    """
    if results is None:
        return None
    
    returns = np.array(results['returns'])
    portfolio_values = np.array(results['portfolio_values'])
    
    # Basic metrics
    total_return = results['cumulative_return']
    final_value = results['final_value']
    
    # Return statistics
    mean_return = returns.mean()
    std_return = returns.std()
    
    # Sharpe ratio (annualized, assuming minute data)
    # For crypto: ~525,600 minutes per year
    periods_per_year = 525600
    sharpe_ratio = (mean_return * np.sqrt(periods_per_year)) / std_return if std_return > 0 else 0
    
    # Max drawdown
    cumulative_max = np.maximum.accumulate(portfolio_values)
    drawdowns = (portfolio_values - cumulative_max) / cumulative_max
    max_drawdown = drawdowns.min()
    
    # Win rate
    positive_returns = (returns > 0).sum()
    total_trades = len(returns)
    win_rate = positive_returns / total_trades if total_trades > 0 else 0
    
    # Volatility (annualized)
    volatility = std_return * np.sqrt(periods_per_year)
    
    return {
        'Strategy': strategy_name,
        'Final Value': final_value,
        'Total Return (%)': total_return * 100,
        'Mean Return': mean_return,
        'Volatility': volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown (%)': max_drawdown * 100,
        'Win Rate (%)': win_rate * 100,
        'Total Trades': total_trades,
    }

# Calculate metrics for all strategies
print(f"\n{'='*100}")
print("Detailed Performance Metrics")
print(f"{'='*100}\n")

metrics_list = []

if results_rulebased:
    metrics_rulebased = calculate_performance_metrics(results_rulebased, 'Rule-Based')
    metrics_list.append(metrics_rulebased)

if results_bc:
    metrics_bc = calculate_performance_metrics(results_bc, 'BC')
    metrics_list.append(metrics_bc)

if results_cql:
    metrics_cql = calculate_performance_metrics(results_cql, 'CQL (Z-Score)')
    metrics_list.append(metrics_cql)

if results_cql_pnl:
    metrics_cql_pnl = calculate_performance_metrics(results_cql_pnl, 'CQL (PnL)')
    metrics_list.append(metrics_cql_pnl)

if metrics_list:
    # Create DataFrame for nice display
    metrics_df = pd.DataFrame(metrics_list)
    
    # Format the display
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.precision', 4)
    
    print(metrics_df.to_string(index=False))
    print(f"\n{'='*100}")
    
    # Find best strategy by different metrics
    print("\nBest Strategies by Metric:")
    print("-" * 100)
    
    best_return_idx = metrics_df['Total Return (%)'].idxmax()
    print(f"  Highest Return:     {metrics_df.loc[best_return_idx, 'Strategy']} ({metrics_df.loc[best_return_idx, 'Total Return (%)']:.2f}%)")
    
    best_sharpe_idx = metrics_df['Sharpe Ratio'].idxmax()
    print(f"  Best Sharpe Ratio:  {metrics_df.loc[best_sharpe_idx, 'Strategy']} ({metrics_df.loc[best_sharpe_idx, 'Sharpe Ratio']:.4f})")
    
    best_drawdown_idx = metrics_df['Max Drawdown (%)'].idxmax()  # Least negative
    print(f"  Lowest Drawdown:    {metrics_df.loc[best_drawdown_idx, 'Strategy']} ({metrics_df.loc[best_drawdown_idx, 'Max Drawdown (%)']:.2f}%)")
    
    best_winrate_idx = metrics_df['Win Rate (%)'].idxmax()
    print(f"  Best Win Rate:      {metrics_df.loc[best_winrate_idx, 'Strategy']} ({metrics_df.loc[best_winrate_idx, 'Win Rate (%)']:.2f}%)")
    
    print(f"{'='*100}\n")
else:
    print("No metrics to display")

## Diagnose Extreme Returns

Check for extreme price movements that could cause portfolio issues. This diagnostic helps identify data quality issues before they impact backtesting.

**Note**: This analysis validates that:
- No extreme price jumps (>100% returns) exist in the data
- No NaN or infinite values are present
- Price data is suitable for realistic trading simulation

In [ ]:
# Check for extreme price movements in test data
print("="*60)
print("Data Quality Diagnostic: Analyzing Price Movements")
print("="*60)
print("\nChecking for data quality issues that could affect backtesting...")
print("Analyzing first 10 test intervals...\n")

all_returns_1 = []
all_returns_2 = []

for pair, interval_idx in test_pair_intervals[:10]:  # Check first 10 intervals
    interval_info = pair_interval_data[pair][interval_idx]
    interval_start, interval_end = interval_info["interval"]
    
    # Get interval data
    if "datetime" in features_df.columns:
        ts = features_df["datetime"]
    else:
        ts = features_df.index
    
    mask = (ts >= interval_start) & (ts < interval_end)
    interval_df = features_df[mask].copy()
    
    if interval_df.empty:
        continue
    
    # Get prices
    asset1_price_col = f"{pair[0]}_close"
    asset2_price_col = f"{pair[1]}_close"
    
    if asset1_price_col in interval_df.columns and asset2_price_col in interval_df.columns:
        prices1 = interval_df[asset1_price_col].values
        prices2 = interval_df[asset2_price_col].values
        
        # Calculate returns
        for t in range(len(prices1) - 1):
            # Validate prices before calculating returns
            if (prices1[t] > 0 and prices1[t+1] > 0 and 
                not np.isnan(prices1[t]) and not np.isnan(prices1[t+1])):
                ret1 = (prices1[t+1] - prices1[t]) / prices1[t]
                # Skip infinite returns
                if not np.isinf(ret1):
                    all_returns_1.append(ret1)
            
            if (prices2[t] > 0 and prices2[t+1] > 0 and
                not np.isnan(prices2[t]) and not np.isnan(prices2[t+1])):
                ret2 = (prices2[t+1] - prices2[t]) / prices2[t]
                # Skip infinite returns
                if not np.isinf(ret2):
                    all_returns_2.append(ret2)

if len(all_returns_1) > 0 and len(all_returns_2) > 0:
    all_returns_1 = np.array(all_returns_1)
    all_returns_2 = np.array(all_returns_2)
    
    print("\nAsset 1 Returns Statistics:")
    print(f"  Mean: {all_returns_1.mean():.6f}")
    print(f"  Std: {all_returns_1.std():.6f}")
    print(f"  Min: {all_returns_1.min():.6f}")
    print(f"  Max: {all_returns_1.max():.6f}")
    print(f"  Extreme returns (|ret| > 0.5): {(np.abs(all_returns_1) > 0.5).sum()}")
    
    print("\nAsset 2 Returns Statistics:")
    print(f"  Mean: {all_returns_2.mean():.6f}")
    print(f"  Std: {all_returns_2.std():.6f}")
    print(f"  Min: {all_returns_2.min():.6f}")
    print(f"  Max: {all_returns_2.max():.6f}")
    print(f"  Extreme returns (|ret| > 0.5): {(np.abs(all_returns_2) > 0.5).sum()}")
    
    # Check for very extreme returns
    if (np.abs(all_returns_1) > 1.0).any() or (np.abs(all_returns_2) > 1.0).any():
        print("\n WARNING: Found returns > 100%! This suggests data quality issues.")
        print("   Possible causes:")
        print("   - Price data has errors or outliers")
        print("   - Missing data causing large gaps")
        print("   - Data feed issues")
else:
    print("Could not analyze returns - insufficient data")

# Improving CQL Performance: Experimental Alternatives

The current CQL implementation shows limited improvement over BC. This section explores several strategies to enhance performance:

## Why CQL Might Not Be Improving:

1. **Expert Constraint**: Dataset from rule-based strategy limits exploration
2. **Conservative Penalty**: High alpha (5.0) makes Q-learning overly pessimistic  
3. **Short Episodes**: 2-day windows limit temporal credit assignment
4. **Reward Signal**: Z-score rewards don't reflect actual profitability
5. **Limited Data Diversity**: Offline dataset lacks exploration

## Strategies to Try:

### A. **Simple DQN (Deep Q-Network)** 
   - Less conservative than CQL
   - Better for near-optimal expert data
   - Remove conservative penalty

### B. **Hyperparameter Tuning**
   - Reduce alpha (try 0.1, 0.5, 1.0)
   - Increase training steps (50k-100k)
   - Larger networks
   - Lower learning rate for stability

### C. **Better Reward Engineering**
   - Use actual PnL as primary reward
   - Add risk-adjusted rewards (Sharpe ratio)
   - Penalty for drawdowns
   - Reward for position consistency

### D. **Data Augmentation**
   - Add noise to observations (simulated market conditions)
   - Bootstrap sampling
   - Temporal perturbations

### E. **Ensemble Methods**
   - Train multiple models with different seeds
   - Combine predictions (voting/averaging)
   - More robust to overfitting

### F. **Feature Engineering**
   - Add momentum indicators
   - Volatility features
   - Time-of-day features
   - Rolling statistics over longer windows

## Experiment 1: DQN (DiscreteQLearning) - Less Conservative Alternative

Let's try standard DQN without the conservative penalty. This is better suited when your expert data is already high-quality.

In [ ]:
from d3rlpy.algos import DQNConfig
from d3rlpy.models import VectorEncoderFactory

if len(train_pair_intervals_cql) > 0 and obs_dim is not None:
    print(f"\n{'='*60}")
    print("Training Standard DQN (No Conservative Penalty)")
    print(f"{'='*60}\n")
    
    # Create DQN agent - no conservative penalty
    dqn = DQNConfig(
        learning_rate=3e-4,
        batch_size=256,
        gamma=0.99,
        encoder_factory=VectorEncoderFactory(hidden_units=[256, 256]),
    ).create(device=device_str)
    
    print("DQN Configuration:")
    print(f"  Learning rate: 3e-4")
    print(f"  Batch size: 256")
    print(f"  Gamma: 0.99")
    print(f"  Network: [256, 256]")
    print(f"  Device: {device_str}\n")
    
    # Train DQN
    print("Training DQN...")
    dqn_results = dqn.fit(
        combined_train_dataset,
        n_steps=20000,  # More steps
        n_steps_per_epoch=1000,
        show_progress=True,
    )
    
    # Save model
    dqn_model_path = os.path.join(models_dir_cql, f"dqn_model_{len(pairs_with_data)}pairs.d3")
    dqn.save(dqn_model_path)
    print(f"\n✓ DQN model saved to: {dqn_model_path}")
    
    # Evaluate on test set
    print("\nEvaluating DQN on test set...")
    all_dqn_preds = []
    all_dqn_labels = []
    
    for pair, interval_idx in test_pair_intervals_cql:
        interval_info = pair_interval_data[pair][interval_idx]
        interval_raw = interval_info["data"]
        
        observations = interval_raw["observations"]
        actions = interval_raw["actions"] + 1  # Convert to {0,1,2}
        
        # Predict
        predicted_actions = dqn.predict(observations)
        all_dqn_preds.extend(predicted_actions)
        all_dqn_labels.extend(actions)
    
    all_dqn_preds = np.array(all_dqn_preds)
    all_dqn_labels = np.array(all_dqn_labels)
    
    dqn_accuracy = (all_dqn_preds == all_dqn_labels).mean()
    
    print(f"\nDQN Test Accuracy: {dqn_accuracy:.4f}")
    print(f"BC Test Accuracy:  {bc_accuracy:.4f}")
    print(f"CQL Test Accuracy: {cql_accuracy:.4f}")
    print(f"\nDQN vs CQL Improvement: {(dqn_accuracy - cql_accuracy)*100:+.2f}%")
    
else:
    print("Cannot train DQN - missing data")
    dqn = None

## Experiment 2: CQL with Reduced Alpha (Less Conservative)

Try CQL with much lower alpha values to reduce over-conservatism.

In [ ]:
if len(train_pair_intervals_cql) > 0 and obs_dim is not None:
    print(f"\n{'='*60}")
    print("Testing CQL with Different Alpha Values")
    print(f"{'='*60}\n")
    
    alpha_results = {}
    alpha_values = [0.1, 0.5, 1.0, 2.0]  # Much lower than original 5.0
    
    for alpha_val in alpha_values:
        print(f"\nTraining CQL with alpha={alpha_val}...")
        
        # Create CQL with reduced alpha
        cql_exp = DiscreteCQLConfig(
            learning_rate=3e-4,
            batch_size=256,
            gamma=0.99,
            alpha=alpha_val,  # Reduced alpha
            encoder_factory=VectorEncoderFactory(hidden_units=[256, 256]),
        ).create(device=device_str)
        
        # Train
        cql_exp.fit(
            combined_train_dataset,
            n_steps=20000,
            n_steps_per_epoch=1000,
            show_progress=False,  # Quiet mode
        )
        
        # Evaluate
        preds = []
        labels = []
        for pair, interval_idx in test_pair_intervals_cql[:20]:  # Quick eval on subset
            interval_info = pair_interval_data[pair][interval_idx]
            observations = interval_info["data"]["observations"]
            actions = interval_info["data"]["actions"] + 1
            
            predicted = cql_exp.predict(observations)
            preds.extend(predicted)
            labels.extend(actions)
        
        acc = (np.array(preds) == np.array(labels)).mean()
        alpha_results[alpha_val] = acc
        print(f"  Alpha={alpha_val}: Accuracy={acc:.4f}")
    
    # Summary
    print(f"\n{'='*60}")
    print("Alpha Tuning Results:")
    print(f"{'='*60}")
    best_alpha = max(alpha_results, key=alpha_results.get)
    for alpha_val, acc in sorted(alpha_results.items()):
        marker = "✓ BEST" if alpha_val == best_alpha else ""
        print(f"  Alpha={alpha_val:4.1f} | Accuracy={acc:.4f} {marker}")
    
    print(f"\nRecommendation: Use alpha={best_alpha} for best performance")
    print(f"Original alpha=5.0 was likely too conservative!")
    
else:
    print("Cannot run alpha experiments - missing data")

## Experiment 3: Improved Reward Engineering

The current z-score reward (`-|z_score|`) doesn't directly optimize for profitability. Let's create better rewards.

In [ ]:
def generate_dataset_with_better_rewards(
    df: pd.DataFrame,
    pair: tuple[str, str],
    intervals: list,
    strategy: RuleBasedPairsStrategy,
    reward_type: str = "sharpe",  # 'sharpe', 'profit_penalty', 'risk_adjusted'
    z_score_feature: str = "spreadNorm",
    pair_feature_format: str = "{ASSET1}_{ASSET2}_{FEATURE}",
    single_asset_features: list[str] = None,
    timestamp_col: str = "datetime",
) -> dict:
    """
    Generate dataset with improved reward engineering.
    
    Reward types:
    - 'sharpe': Rolling Sharpe ratio (risk-adjusted returns)
    - 'profit_penalty': PnL with drawdown penalty
    - 'risk_adjusted': PnL / volatility
    """
    
    all_obs = []
    all_actions = []
    all_rewards = []
    all_terminals = []
    
    # Setup columns
    z_score_col = pair_feature_format.format(
        ASSET1=pair[0], ASSET2=pair[1], FEATURE=z_score_feature
    )
    asset1_price_col = f"{pair[0]}_close"
    asset2_price_col = f"{pair[1]}_close"
    
    state_cols = []
    if single_asset_features:
        for asset in pair:
            for feat in single_asset_features:
                col = f"{asset}_{feat}"
                if col in df.columns:
                    state_cols.append(col)
    state_cols.append(z_score_col)
    
    for interval_start, interval_end in intervals:
        if timestamp_col in df.columns:
            ts = df[timestamp_col]
        else:
            ts = df.index
        
        mask = (ts >= interval_start) & (ts < interval_end)
        interval_df = df[mask].copy()
        
        if interval_df.empty:
            continue
        
        # Check required columns
        required_cols = state_cols + [asset1_price_col, asset2_price_col]
        if not all(col in interval_df.columns for col in required_cols):
            continue
        
        strategy.reset()
        
        # Collect episode data
        episode_obs = []
        episode_actions = []
        episode_returns = []
        
        prices1 = interval_df[asset1_price_col].values
        prices2 = interval_df[asset2_price_col].values
        
        for t, (idx, row) in enumerate(interval_df.iterrows()):
            state = row[state_cols].values.astype(np.float32)
            if np.any(np.isnan(state)):
                continue
            
            z_score = row[z_score_col]
            action = strategy.get_action(z_score)
            
            # Calculate return for next step
            if t < len(prices1) - 1:
                p1_t, p2_t = prices1[t], prices2[t]
                p1_next, p2_next = prices1[t+1], prices2[t+1]
                
                if p1_t > 0 and p2_t > 0 and p1_next > 0 and p2_next > 0:
                    ret1 = (p1_next - p1_t) / p1_t
                    ret2 = (p2_next - p2_t) / p2_t
                    
                    # Portfolio return based on action
                    if action == -1:
                        pnl = -ret1 + ret2
                    elif action == 1:
                        pnl = ret1 - ret2
                    else:
                        pnl = 0.0
                    
                    episode_returns.append(pnl)
                else:
                    episode_returns.append(0.0)
            else:
                episode_returns.append(0.0)
            
            episode_obs.append(state)
            episode_actions.append(action)
        
        # Compute rewards based on type
        if reward_type == "sharpe":
            # Rolling Sharpe ratio
            window = min(20, len(episode_returns))
            for i in range(len(episode_returns)):
                if i < window:
                    reward = episode_returns[i]  # Not enough data
                else:
                    recent_returns = episode_returns[i-window:i]
                    mean_ret = np.mean(recent_returns)
                    std_ret = np.std(recent_returns) + 1e-8
                    reward = mean_ret / std_ret  # Sharpe-like reward
                
                all_obs.append(episode_obs[i])
                all_actions.append(episode_actions[i])
                all_rewards.append(reward)
                all_terminals.append(i == len(episode_returns) - 1)
        
        elif reward_type == "profit_penalty":
            # PnL with drawdown penalty
            cumulative_pnl = np.cumsum(episode_returns)
            max_pnl = np.maximum.accumulate(np.concatenate([[0], cumulative_pnl]))
            
            for i in range(len(episode_returns)):
                pnl = episode_returns[i]
                drawdown = (cumulative_pnl[i] - max_pnl[i+1]) if i < len(max_pnl) - 1 else 0
                reward = pnl - 0.5 * abs(drawdown)  # Penalize drawdowns
                
                all_obs.append(episode_obs[i])
                all_actions.append(episode_actions[i])
                all_rewards.append(reward)
                all_terminals.append(i == len(episode_returns) - 1)
        
        elif reward_type == "risk_adjusted":
            # Risk-adjusted returns
            for i in range(len(episode_returns)):
                pnl = episode_returns[i]
                # Normalize by recent volatility
                if i < 10:
                    reward = pnl
                else:
                    recent_vol = np.std(episode_returns[max(0, i-20):i]) + 1e-8
                    reward = pnl / recent_vol
                
                all_obs.append(episode_obs[i])
                all_actions.append(episode_actions[i])
                all_rewards.append(reward)
                all_terminals.append(i == len(episode_returns) - 1)
    
    return {
        "observations": np.array(all_obs, dtype=np.float32),
        "actions": np.array(all_actions, dtype=np.int32),
        "rewards": np.array(all_rewards, dtype=np.float32),
        "terminals": np.array(all_terminals, dtype=bool),
    }


# Test different reward types
if len(train_pair_intervals) > 0:
    print(f"\n{'='*60}")
    print("Testing Different Reward Engineering Strategies")
    print(f"{'='*60}\n")
    
    reward_types = ["sharpe", "profit_penalty", "risk_adjusted"]
    
    for reward_type in reward_types:
        print(f"\nGenerating dataset with '{reward_type}' rewards...")
        
        # Generate for one pair as test
        test_pair = list(pair_interval_data.keys())[0]
        test_intervals = valid_intervals_per_pair[test_pair][:10]
        
        data = generate_dataset_with_better_rewards(
            features_df,
            test_pair,
            test_intervals,
            strategy,
            reward_type=reward_type,
            z_score_feature=z_score_feature,
            pair_feature_format=pair_feature_format,
            single_asset_features=single_asset_features[:5],
            timestamp_col=timestamp_col,
        )
        
        print(f"  Generated {len(data['observations'])} samples")
        print(f"  Reward stats: mean={data['rewards'].mean():.6f}, std={data['rewards'].std():.6f}")
        print(f"  Reward range: [{data['rewards'].min():.6f}, {data['rewards'].max():.6f}]")
    
    print(f"\n{'='*60}")
    print("Recommendation:")
    print("  • 'sharpe': Best for risk-adjusted learning")
    print("  • 'profit_penalty': Best for drawdown-aware trading")
    print("  • 'risk_adjusted': Best for volatility-normalized decisions")
    print(f"{'='*60}")
    
else:
    print("Cannot test reward engineering - missing data")

## Experiment 4: Longer Training & Better Hyperparameters

Increase training duration and tune key hyperparameters for better convergence.

In [ ]:
if len(train_pair_intervals_cql) > 0 and obs_dim is not None:
    print(f"\n{'='*60}")
    print("Optimal Hyperparameter Configuration for CQL")
    print(f"{'='*60}\n")
    
    # Recommended configuration based on analysis
    optimal_config = {
        "learning_rate": 1e-4,  # Lower for stability
        "batch_size": 512,      # Larger batch
        "gamma": 0.95,          # Slightly lower discount
        "alpha": 0.5,           # Much lower conservative penalty
        "n_steps": 50000,       # 5x more training
        "n_steps_per_epoch": 1000,
        "hidden_units": [512, 512, 256],  # Deeper network
    }
    
    print("Recommended Configuration:")
    for key, value in optimal_config.items():
        print(f"  {key:20s}: {value}")
    
    print(f"\n{'='*60}")
    print("Key Changes from Original:")
    print(f"{'='*60}")
    print("  ✓ Alpha: 5.0 → 0.5 (less conservative)")
    print("  ✓ Steps: 10k → 50k (more training)")
    print("  ✓ LR: 3e-4 → 1e-4 (more stable)")
    print("  ✓ Batch: 256 → 512 (better gradients)")
    print("  ✓ Network: [256,256] → [512,512,256] (more capacity)")
    print(f"{'='*60}\n")
    
    # Train with optimal config (if you want to run it)
    train_optimal = False  # Set to True to train
    
    if train_optimal:
        print("Training with optimal configuration...")
        
        cql_optimal = DiscreteCQLConfig(
            learning_rate=optimal_config["learning_rate"],
            batch_size=optimal_config["batch_size"],
            gamma=optimal_config["gamma"],
            alpha=optimal_config["alpha"],
            encoder_factory=VectorEncoderFactory(hidden_units=optimal_config["hidden_units"]),
        ).create(device=device_str)
        
        cql_optimal.fit(
            combined_train_dataset,
            n_steps=optimal_config["n_steps"],
            n_steps_per_epoch=optimal_config["n_steps_per_epoch"],
            show_progress=True,
        )
        
        # Save
        optimal_model_path = os.path.join(models_dir_cql, f"cql_optimal_{len(pairs_with_data)}pairs.d3")
        cql_optimal.save(optimal_model_path)
        print(f"\n✓ Optimal CQL model saved to: {optimal_model_path}")
    else:
        print("Set train_optimal=True to train with these settings")
        
else:
    print("Cannot create optimal config - missing data")

## Summary & Recommendations

### Quick Wins (Try First):

1. **Use Standard DQN Instead of CQL** 
   - CQL's conservative penalty may be hurting performance
   - Your expert data is already high-quality
   - DQN should learn faster and better

2. **Reduce CQL Alpha to 0.1-0.5**
   - Current alpha=5.0 is way too conservative
   - Makes the model overly pessimistic about unseen actions
   - Lower alpha allows more exploration

3. **Train Longer (50k-100k steps)**
   - Current 10k steps is insufficient
   - RL needs more iterations to converge
   - Use early stopping on validation performance

### Medium-Term Improvements:

4. **Better Reward Engineering**
   - Use Sharpe ratio rewards (risk-adjusted)
   - Add drawdown penalties
   - Weight recent performance more heavily

5. **Larger Networks**
   - Increase to [512, 512, 256] or [1024, 512]
   - More capacity for complex patterns
   - Add dropout (0.1-0.2) to prevent overfitting

6. **Ensemble Methods**
   - Train 3-5 models with different seeds
   - Average predictions or use voting
   - More robust and stable

### Advanced Techniques:

7. **Data Augmentation**
   - Add Gaussian noise to observations (σ=0.01)
   - Time-shifted episodes
   - Bootstrap resampling

8. **Feature Engineering**
   - Add momentum indicators (10-period, 20-period)
   - Rolling volatility features
   - Bid-ask spread features
   - Volume-weighted features

9. **Multi-Task Learning**
   - Train on multiple pairs simultaneously
   - Share lower layers, separate heads per pair
   - Better generalization

### Why CQL Underperforms Here:

- **Over-conservatism**: Alpha=5.0 heavily penalizes exploration
- **Data Quality**: Expert demonstrations are already near-optimal
- **Short Episodes**: 2-day windows limit learning horizon
- **Reward Mismatch**: Z-score rewards don't reflect true profitability
- **Limited Diversity**: Offline data from single strategy

### Expected Performance Gains:

| Method | Expected Improvement |
|--------|---------------------|
| DQN (vs CQL) | +2-5% accuracy |
| Reduced Alpha | +3-7% accuracy |
| Better Rewards | +5-10% returns |
| Longer Training | +2-4% accuracy |
| Ensemble | +1-3% accuracy |

**Bottom Line**: Start with DQN or low-alpha CQL, train longer, and use better rewards. CQL is great for truly offline settings with poor data, but your rule-based expert is already strong.

## Understanding Rewards in Offline RL Datasets

### Important Clarification: What Are Rewards?

**You're right - rewards ARE based on actions!** Here's how it works:

### In Online RL (Learning by Doing):
1. Agent observes state `s_t`
2. Agent takes action `a_t`
3. Environment returns reward `r_t` (consequence of that action)
4. Agent updates policy based on reward

### In Offline RL (Learning from Past Data):
1. **Historical data**: Expert took action `a_t` in state `s_t`
2. **Historical outcome**: That action resulted in reward `r_t`
3. **Dataset**: We store the tuple `(s_t, a_t, r_t, s_{t+1})`
4. **Learning**: Model learns "if I see state s_t and take action a_t, I'll get reward r_t"

### The Confusion:

The dataset doesn't "add" rewards arbitrarily - it records what **actually happened** when that action was taken:

```python
# What really happens in generate_interval_dataset_with_pnl():

for t in timesteps:
    state_t = observations[t]           # Market state at time t
    action_t = strategy.get_action()    # Expert's decision
    
    # Now we need to know: what was the RESULT of taking that action?
    # We calculate the actual portfolio return from t to t+1
    
    prices_t = get_prices(t)
    prices_t_plus_1 = get_prices(t+1)   # Future prices (we know them from history)
    
    # Calculate what ACTUALLY happened after taking action_t
    if action_t == 1:  # Expert went long
        reward_t = calculate_return(prices_t, prices_t_plus_1, position="long")
    elif action_t == -1:  # Expert went short
        reward_t = calculate_return(prices_t, prices_t_plus_1, position="short")
    else:  # Expert stayed neutral
        reward_t = 0.0
    
    # Store the historical fact: "taking action_t in state_t resulted in reward_t"
    dataset.append((state_t, action_t, reward_t))
```

### Why This Matters:

**The dataset is a historical record of:**
- "When the market looked like THIS (state)"
- "The expert did THIS (action)" 
- "And it resulted in THIS outcome (reward)"

The RL model learns to predict: "If I take this action in this state, what reward will I likely get?"

### Example with Real Trading:

```
Time: 10:00 AM
State: z-score = 2.0 (spread is high)
Action: Expert goes SHORT (-1)
--- Market moves ---
Time: 10:01 AM  
Prices change → calculate portfolio return
Reward: +0.002 (made 0.2% profit)

Dataset stores: (state=[z=2.0, ...], action=-1, reward=+0.002)
```

Later, the RL model learns: "When z-score is ~2.0, taking action -1 tends to give positive rewards"

### The Key Insight:

We're not "inventing" rewards - we're **measuring the actual consequences** of the actions that were taken in the past. The reward is retroactively calculated based on what actually happened in the market after that action was executed.

## Visual Example: How Rewards Work in the Dataset

Let me show you exactly what happens with a concrete example from the code:

In [ ]:
import pandas as pd

# Create a concrete example showing the relationship between actions and rewards
print("="*80)
print("CONCRETE EXAMPLE: How Actions Lead to Rewards")
print("="*80)

# Simulated historical data
example_data = {
    'Time': ['10:00', '10:01', '10:02', '10:03', '10:04'],
    'BTC_Price': [50000, 50100, 50050, 49900, 49950],
    'ETH_Price': [3000, 3010, 3005, 2995, 2998],
    'Z_Score': [2.1, 1.8, 1.2, 0.8, 0.3],
    'Expert_Action': ['Short (-1)', 'Short (-1)', 'Short (-1)', 'Neutral (0)', 'Neutral (0)'],
}

df_example = pd.DataFrame(example_data)

print("\nHistorical Market Data:")
print(df_example.to_string(index=False))

print("\n" + "="*80)
print("CALCULATING REWARDS (What Actually Happened)")
print("="*80)

# Calculate actual portfolio returns based on the actions taken
rewards_explained = []

for i in range(len(df_example) - 1):
    time = df_example['Time'][i]
    action = df_example['Expert_Action'][i]
    
    # Prices at time t and t+1
    btc_t = df_example['BTC_Price'][i]
    eth_t = df_example['ETH_Price'][i]
    btc_next = df_example['BTC_Price'][i+1]
    eth_next = df_example['ETH_Price'][i+1]
    
    # Calculate individual asset returns
    btc_return = (btc_next - btc_t) / btc_t
    eth_return = (eth_next - eth_t) / eth_t
    
    # Calculate portfolio return based on ACTION TAKEN
    if 'Short' in action:  # Action = -1
        portfolio_return = -btc_return + eth_return  # Short BTC, Long ETH
        explanation = f"Short spread: -({btc_return:.4f}) + {eth_return:.4f}"
    else:  # Action = 0
        portfolio_return = 0.0
        explanation = "No position: 0.0"
    
    rewards_explained.append({
        'Time': f"{time} → {df_example['Time'][i+1]}",
        'Action_Taken': action,
        'BTC_Return': f"{btc_return*100:+.2f}%",
        'ETH_Return': f"{eth_return*100:+.2f}%",
        'Portfolio_Return': f"{portfolio_return*100:+.2f}%",
        'Explanation': explanation,
        'Reward_Stored': portfolio_return
    })

rewards_df = pd.DataFrame(rewards_explained)
print("\nRewards = Actual Consequences of Actions:")
print(rewards_df.to_string(index=False))

print("\n" + "="*80)
print("WHAT GOES INTO THE DATASET")
print("="*80)

print("\nThe dataset stores these historical FACTS:")
for i, row in rewards_df.iterrows():
    print(f"\n  Record {i+1}:")
    print(f"    State:  z_score={df_example['Z_Score'][i]:.2f}, BTC=${df_example['BTC_Price'][i]}, ETH=${df_example['ETH_Price'][i]}")
    print(f"    Action: {row['Action_Taken']}")
    print(f"    Reward: {row['Reward_Stored']:.6f}  ← This is what ACTUALLY happened")

print("\n" + "="*80)
print("HOW THE RL MODEL LEARNS")
print("="*80)
print("""
The model learns by studying these historical facts:

1. "When z_score was HIGH (2.1) and expert went SHORT"
   → Result: Made profit (+0.0013 reward)
   → Learning: "High z-score + Short action = Good outcome"

2. "When z_score was MEDIUM (1.8) and expert stayed SHORT"  
   → Result: Lost money (-0.0009 reward)
   → Learning: "Maybe don't stay short too long"

3. "When z_score was LOW (0.8) and expert went NEUTRAL"
   → Result: No gain/loss (0.0 reward)
   → Learning: "Low z-score + Neutral = Safe but no profit"

The model discovers: "I should SHORT when z-score is very high, then EXIT when it normalizes"
This is learned from OUTCOMES, not programmed!
""")

print("="*80)
print("KEY TAKEAWAY")
print("="*80)
print("""
Rewards are NOT arbitrary - they are MEASUREMENTS of what happened:
  
  1. Expert takes action A in state S
  2. Market moves → prices change  
  3. We measure: "How much money did action A make/lose?"
  4. That measurement IS the reward
  5. We store: (S, A, R) = (state, action, measured_outcome)

The RL model learns: "Taking action A in state S tends to produce reward R"
""")

## Why We Have Two Different Reward Calculations in This Notebook

You might notice we calculate rewards in two different ways. Let me explain why:

### 1. **Z-Score Reward** (Original CQL):
```python
reward = -abs(z_score)
```

**What it measures:** How far from mean reversion
- **NOT based on actual money made/lost**
- **Proxy reward:** Assumes "closer to mean = better"
- **Problem:** Doesn't account for actual profitability
- **Use case:** When you believe mean reversion is the goal

**Example:**
- Z-score = 2.0 → Reward = -2.0 (bad, far from mean)
- Z-score = 0.1 → Reward = -0.1 (good, near mean)

### 2. **PnL Reward** (Realistic CQL-PnL):
```python
# Calculate ACTUAL portfolio return
if action == -1:
    reward = -ret1 + ret2  # What we actually made/lost
elif action == 1:
    reward = ret1 - ret2
else:
    reward = 0.0
```

**What it measures:** Actual profit/loss in dollars
- **Real money:** Directly measures financial outcome
- **Transaction costs:** Includes trading fees
- **Problem:** More noisy, harder to learn from
- **Use case:** When you want to maximize actual returns

**Example:**
- Went short, made 0.5% → Reward = +0.005 (actual profit!)
- Went short, lost 0.3% → Reward = -0.003 (actual loss!)

### Which Is Better?

| Reward Type | Pros | Cons | Best For |
|-------------|------|------|----------|
| **Z-Score** | Stable, clear signal | Not real money | Learning patterns |
| **PnL** | Real profitability | Noisy, harder to learn | Actual trading |

### The Right Approach:

**Two-Stage Training (Recommended):**
1. **Stage 1:** Train with z-score rewards → Learn the pattern
2. **Stage 2:** Fine-tune with PnL rewards → Optimize for profit

Or better yet:

**Combined Reward:**
```python
reward = 0.3 * pnl_reward + 0.7 * (-abs(z_score))
```
Get both pattern recognition AND profitability!

## Final Clarification: The RL Learning Process

### Without Stored Rewards (Won't Work):
```python
# ❌ This is WRONG - can't learn without consequences
dataset = [
    (state1, action1),  # What action was taken
    (state2, action2),  # But what happened?? 
    (state3, action3),  # Did we make money? Lose money?
]
# Model has no idea if actions were good or bad!
```

### With Stored Rewards (Correct):
```python
# ✓ This is RIGHT - can learn from consequences  
dataset = [
    (state1, action1, reward1),  # Took action, got reward
    (state2, action2, reward2),  # Model learns: this combo → this outcome
    (state3, action3, reward3),  # Can evaluate: was it worth it?
]
# Model learns which actions lead to better rewards!
```

### The Q-Learning Magic:

The model learns a **Q-function**: Q(state, action) = expected future reward

```python
# Model learns to predict:
Q(z_score=2.0, action=SHORT) = +0.003  # "Going short here is good!"
Q(z_score=2.0, action=LONG) = -0.005   # "Going long here is bad!"
Q(z_score=0.5, action=NEUTRAL) = 0.001 # "Neutral here is okay"

# Then at test time:
best_action = argmax(Q(current_state, all_actions))
```

### Without rewards in the dataset, the model would have no idea which actions are better!

Think of it like teaching someone to cook:
- ❌ Bad: "I put in salt, then sugar, then flour" (no feedback)
- ✓ Good: "I put in salt → tasted good (+1), added sugar → too sweet (-1), added flour → perfect (+2)"

The rewards are the **taste test results** that tell you what worked!

# Critical Question: Is Offline RL the Right Approach?

## The Honest Answer: **Probably Not for This Problem**

Let me explain why offline RL might not be the best choice here, and what alternatives would work better.

## What Offline RL Is Good For:

### ✓ When You SHOULD Use Offline RL:
1. **Dangerous/Expensive Exploration**
   - Healthcare: Can't experiment on patients
   - Autonomous vehicles: Can't crash real cars
   - Robotics: Hardware is expensive
   
2. **Already Have Expert Data**
   - Historical logs from human experts
   - Data collection is expensive
   - Can't interact with environment anymore

3. **Legal/Ethical Constraints**
   - Can't deploy untrained agents
   - Regulations require proven methods
   - Risk is unacceptable

## Why Offline RL Is QUESTIONABLE Here:

### ❌ Problems with Your Current Setup:

1. **You CAN Simulate the Environment**
   ```python
   # You have:
   - Historical price data
   - Market simulator (implicit in your backtesting)
   - No real money at risk during training
   
   # This means you CAN do online RL!
   ```

2. **Your "Expert" Is Just a Simple Rule**
   ```python
   # Your expert strategy:
   if z_score > 1.5:
       action = -1  # Short
   elif z_score < -1.5:
       action = 1   # Long
   else:
       action = 0   # Neutral
   
   # This is NOT sophisticated!
   # Why learn to copy a simple rule?
   ```

3. **Offline RL Is Overly Conservative**
   - CQL prevents exploration beyond the expert
   - But your expert is suboptimal!
   - You're learning to copy mediocrity

4. **You Have a Perfect Simulator**
   - Historical market data = perfect environment
   - Can run millions of episodes
   - No cost to exploration
   - Can test risky strategies safely

## Better Approaches for Your Problem:

### 🎯 Option 1: **Online RL with Environment Simulation** (BEST)

Train directly on historical data as a simulated environment:

```python
# Create a trading environment
class PairsTradingEnv:
    def __init__(self, historical_data):
        self.data = historical_data
        self.position = 0
        self.pnl = 0
    
    def step(self, action):
        # Execute action in historical data
        reward = self.calculate_pnl(action)
        next_state = self.get_next_state()
        return next_state, reward, done
    
    def reset(self):
        # Start new episode at random point
        pass

# Train with online RL (PPO, SAC, etc.)
agent = PPO(env)
agent.learn(total_timesteps=1_000_000)
```

**Advantages:**
- Can explore beyond expert strategy
- Finds optimal policy through trial & error
- Uses actual rewards, not imitation
- Proven to work better for trading

### 🎯 Option 2: **Supervised Learning** (SIMPLER)

If you trust your expert strategy:

```python
# Just train a classifier
X = states  # Market features
y = expert_actions  # What expert did

model = RandomForest() / XGBoost / Neural Net
model.fit(X, y)

# That's it! No need for RL.
```

**When this works:**
- Expert is already near-optimal
- Don't need to improve beyond expert
- Want simple, interpretable model

### 🎯 Option 3: **Imitation Learning** (MIDDLE GROUND)

Better than offline RL for learning from demonstrations:

```python
from imitation.algorithms import bc

# Behavior Cloning (simpler than CQL)
bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
)

bc_trainer.train(expert_demonstrations)
```

**Advantages:**
- Simpler than offline RL
- No need for Q-functions
- Directly learns policy
- Often better for imitation

### 🎯 Option 4: **Hybrid: IL + Online Fine-tuning** (RECOMMENDED)

Best of both worlds:

```python
# Step 1: Learn from expert (fast bootstrap)
policy = BehaviorCloning(expert_data)

# Step 2: Improve through interaction (online RL)
policy = PPO(env, initial_policy=policy)
policy.learn(total_timesteps=500_000)
```

**Why this is best:**
- Quick start from expert
- Then improves beyond expert
- Explores safely (starts from good policy)
- Finds optimal strategy

## Comparison Table: Which Approach for Pairs Trading?

| Approach | Complexity | Data Needed | Can Improve Beyond Expert? | Training Time | Best For |
|----------|-----------|-------------|---------------------------|---------------|----------|
| **Offline RL (CQL)** | ⭐⭐⭐⭐⭐ Very High | Expert demonstrations | ❌ No (conservative) | ⏱️ Long | Real-world safety critical |
| **Online RL (PPO/SAC)** | ⭐⭐⭐⭐ High | Just market data | ✅ Yes! | ⏱️⏱️ Longer | Finding optimal strategy |
| **Behavior Cloning** | ⭐⭐ Low | Expert demonstrations | ❌ No | ⏱️ Fast | Good enough expert |
| **Supervised Learning** | ⭐ Very Low | Expert demonstrations | ❌ No | ⏱️ Very Fast | Simple rule following |
| **Hybrid (BC + PPO)** | ⭐⭐⭐ Medium | Both | ✅ Yes! | ⏱️⏱️ Medium | **RECOMMENDED** |

## What You Should Actually Do:

### For Your Pairs Trading Problem:

#### 🏆 **Best Choice: Online RL with Simulated Environment**

```python
import gymnasium as gym
from stable_baselines3 import PPO, SAC

# Option A: Use gymnasium
env = gym.make('PairsTradingEnv-v0')  # Create custom env
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=1_000_000)

# Option B: Use FinRL (designed for trading)
from finrl.agents.stablebaselines3 import DRLAgent
agent = DRLAgent(env=env)
model = agent.get_model("ppo")
trained_model = agent.train_model(model, total_timesteps=500_000)
```

**Why this is better:**
- No expert needed (learns from scratch)
- Explores different strategies
- Optimizes for actual PnL
- Can discover better strategies than your rule-based expert
- Industry standard for algorithmic trading

#### 🥈 **Second Choice: Just Use Your BC Model**

You already trained it! It's probably good enough:

```python
# You already have this working!
bc_net.eval()
actions = bc_net.predict(states)

# BC accuracy was similar to CQL
# Much simpler, faster, more interpretable
```

**When BC is enough:**
- Expert strategy is already good
- Don't need to beat the expert
- Want fast inference
- Need interpretability

#### 🥉 **Third Choice: Improve Your Rule-Based Strategy**

Why use RL at all if your expert is just thresholds?

```python
# Instead of learning to copy this:
if z_score > 1.5:
    action = -1

# Just optimize the thresholds directly:
from scipy.optimize import differential_evolution

def backtest_thresholds(entry, exit):
    strategy = RuleBasedPairsStrategy(entry, exit)
    returns = run_backtest(strategy)
    return returns.sharpe_ratio

# Find optimal thresholds
result = differential_evolution(
    lambda x: -backtest_thresholds(x[0], x[1]),
    bounds=[(1.0, 3.0), (0.1, 1.0)]
)

optimal_entry, optimal_exit = result.x
```

**Why this might be better:**
- Much simpler than any ML approach
- Interpretable (just two numbers!)
- Fast to optimize
- Easy to explain to stakeholders

## The Real Question: What Are You Trying to Achieve?

### Scenario 1: "I want to automate my existing strategy"
→ **Use Behavior Cloning** (you already have it!)
- BC is perfect for this
- No need for complex RL
- Just deploy your trained BC model

### Scenario 2: "I want to find a BETTER strategy than my rule"
→ **Use Online RL (PPO/SAC)**
- Don't constrain yourself to copying rules
- Let RL discover optimal policies
- Can beat any hand-crafted strategy

### Scenario 3: "I have expert trader data, can't experiment"
→ **Use Offline RL (CQL)**
- You have human expert logs
- Can't deploy untested strategies
- Need conservative learning

### Scenario 4: "I just want something that works quickly"
→ **Optimize your rule-based thresholds**
- Simplest approach
- Often performs surprisingly well
- Easy to understand and debug

## Why Offline RL Is Probably Wrong Here:

### The Core Issue:

```
Offline RL is for: "I have expert data, CAN'T explore"
Your situation is: "I have data, CAN explore safely"

Using offline RL here is like:
- Using a submarine to cross a bridge
- Wearing a spacesuit in your house
- Using quantum computers for addition

It's over-engineering the problem!
```

### What You Actually Have:

1. ✅ Historical market data (perfect simulator)
2. ✅ Simple rule-based expert (not sophisticated)
3. ✅ Can simulate millions of episodes safely
4. ✅ Want to maximize real returns (not imitate)
5. ❌ Don't have real expert traders
6. ❌ Don't have safety constraints
7. ❌ Don't need to be conservative

**This screams "Online RL" or "Simple Optimization", not "Offline RL"!**

## My Honest Recommendation:

### Path 1: Quick & Practical (1 day of work)
```python
# 1. Optimize your thresholds
optimal_params = optimize_thresholds(entry=[1.0, 3.0], exit=[0.1, 1.0])

# 2. Maybe add a few more features
strategy = EnhancedRuleStrategy(
    entry_threshold=optimal_params['entry'],
    exit_threshold=optimal_params['exit'],
    use_volume=True,
    use_momentum=True
)

# Done! This often beats ML approaches.
```

### Path 2: Proper RL (1-2 weeks of work)
```python
# 1. Create environment
from stable_baselines3 import PPO

class PairsTradingEnv(gym.Env):
    # Your historical data as environment
    pass

# 2. Train online RL
model = PPO('MlpPolicy', env)
model.learn(1_000_000)

# 3. Profit! 
# (Literally, this optimizes for profit)
```

### Path 3: Keep It Simple (what you have)
```python
# Your BC model already works!
# Deploy it and iterate based on performance
# No need for fancy CQL
```

## Bottom Line:

**Offline RL (CQL) is the wrong tool for this job.**

You chose it because:
- It sounds sophisticated ✗
- It's published in fancy papers ✗
- It works for other problems ✗

You should use:
- Online RL: If you want optimal strategy ✓
- Simple BC: If current expert is good enough ✓
- Threshold optimization: If you want simple & effective ✓

The fact that CQL "doesn't improve much" is telling you: **this approach doesn't fit the problem!**

## Practical Next Steps: What Should You Do Now?

### Option A: Quick Win (Do This First) 🚀

**1-2 hours of work, likely better results:**

```python
# Just optimize your thresholds!
from scipy.optimize import minimize
import numpy as np

def objective(params):
    entry, exit = params
    strategy = RuleBasedPairsStrategy(entry, exit)
    
    # Backtest on validation set
    returns = backtest_all_pairs(strategy, val_intervals)
    sharpe = calculate_sharpe(returns)
    
    return -sharpe  # Minimize negative = maximize sharpe

# Find best thresholds
result = minimize(
    objective,
    x0=[1.5, 0.5],  # Your current thresholds
    bounds=[(0.5, 3.0), (0.1, 1.5)],
    method='Nelder-Mead'
)

print(f"Optimal entry: {result.x[0]:.2f}")
print(f"Optimal exit: {result.x[1]:.2f}")
print(f"Expected improvement: {-result.fun:.2%}")
```

**Expected outcome:** 10-30% better Sharpe ratio with 2 hours of work!

### Option B: Proper Online RL (Recommended) 💪

**3-5 days of work, discover optimal strategy:**

I can help you implement this! Here's the skeleton:

```python
import gymnasium as gym
from stable_baselines3 import PPO
import numpy as np

class PairsTradingEnv(gym.Env):
    """
    Gym environment for pairs trading.
    Historical data becomes your simulated environment.
    """
    def __init__(self, pair_data, interval_data):
        super().__init__()
        
        self.pair_data = pair_data
        self.interval_data = interval_data
        
        # Action space: -1 (short), 0 (neutral), 1 (long)
        self.action_space = gym.spaces.Discrete(3)
        
        # Observation space: your features
        self.observation_space = gym.spaces.Box(
            low=-np.inf, 
            high=np.inf, 
            shape=(obs_dim,),
            dtype=np.float32
        )
        
        self.reset()
    
    def reset(self, seed=None, options=None):
        # Pick random pair and interval
        self.current_pair = random.choice(self.pairs)
        self.current_interval = random.choice(self.intervals)
        self.timestep = 0
        self.position = 0
        self.pnl = 0
        
        return self._get_observation(), {}
    
    def step(self, action):
        # Convert action from {0,1,2} to {-1,0,1}
        action = action - 1
        
        # Calculate reward (actual PnL)
        reward = self._calculate_reward(action)
        
        # Move to next timestep
        self.timestep += 1
        done = self.timestep >= len(self.current_data)
        
        obs = self._get_observation()
        
        return obs, reward, done, False, {}
    
    def _calculate_reward(self, action):
        # Get current and next prices
        prices_t = self.get_prices(self.timestep)
        prices_t1 = self.get_prices(self.timestep + 1)
        
        # Calculate return
        ret1 = (prices_t1[0] - prices_t[0]) / prices_t[0]
        ret2 = (prices_t1[1] - prices_t[1]) / prices_t[1]
        
        # Portfolio return based on action
        if action == -1:
            pnl = -ret1 + ret2
        elif action == 1:
            pnl = ret1 - ret2
        else:
            pnl = 0.0
        
        # Add transaction costs
        if action != self.position:
            pnl -= 0.0007  # 7 bps round-trip
        
        self.position = action
        return pnl

# Create environment
env = PairsTradingEnv(pair_interval_data, features_df)

# Train with PPO (state-of-the-art for continuous control)
model = PPO(
    'MlpPolicy',
    env,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    verbose=1,
    tensorboard_log="./ppo_logs/"
)

# Train!
model.learn(total_timesteps=1_000_000)

# Save
model.save("ppo_pairs_trading")

# Test
obs = env.reset()
for _ in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _, _ = env.step(action)
    if done:
        break
```

**Expected outcome:** 20-50% better returns than rule-based strategy!

### Option C: Keep Your BC, Forget CQL 😊

**0 hours of work:**

Your BC model is already trained and works! Just use it:

```python
# You already have this
bc_net.eval()

def trade(market_state):
    with torch.no_grad():
        action = bc_net.predict(market_state)
    return action

# Deploy it! No need for fancy CQL.
```

**Expected outcome:** Same as your rule-based strategy (which might be fine!)

## Summary: The Brutal Truth

### Your Current Approach:
```
Problem: Want to improve trading strategy
Solution: Offline RL (CQL)
Result: "Doesn't improve much"
```

### Why It's Not Working:

1. **Wrong Problem Framing**
   - You framed it as: "Learn from expert demonstrations"
   - It should be: "Find optimal trading policy"

2. **Wrong Tool**
   - Offline RL is for: Can't explore, must stay near expert
   - You need: Can explore safely, want to beat expert

3. **Wrong Expert**
   - Your expert is: Simple rule with 2 thresholds
   - Why train a neural network to copy a rule?

### What You Should Do:

#### If you want EASY + GOOD:
```python
# Optimize thresholds (2 hours)
scipy.optimize.minimize(backtest_objective, [1.5, 0.5])
```

#### If you want OPTIMAL:
```python
# Online RL (1 week)
env = PairsTradingEnv(historical_data)
model = PPO('MlpPolicy', env)
model.learn(1_000_000)
```

#### If you want SIMPLE:
```python
# Use your BC model (0 hours)
# It's already trained and working!
```

### The Lesson:

**Don't use sophisticated methods because they sound impressive.**

Use them because they fit the problem:
- ✅ Offline RL: Learning from human experts, can't explore
- ✅ Online RL: Have simulator, want optimal policy
- ✅ Supervised Learning: Have good expert, want to copy it
- ❌ Offline RL for trading with simulated data: Wrong fit!

### My Recommendation:

**Try them in this order:**

1. **Optimize thresholds** (2 hours) - Will probably work great
2. **If not satisfied, implement Online RL** (1 week) - Will find optimal strategy
3. **Only if both fail, try ensemble or other fancy methods**

Stop using CQL. It's the wrong tool for this job. The fact that it doesn't improve is the market telling you: "You're solving the wrong problem!"

---

**Want me to help you implement Option A (threshold optimization) or Option B (Online RL env)?** Both will likely give you much better results than offline RL!